In [1]:
import pandas as pd
import pyodbc
import time 
import numpy as np
from datetime import datetime
print(pyodbc.drivers())

['PostgreSQL', 'MySQL', 'ODBC Driver 18 for SQL Server']


In [2]:
%run SetupDwhColombia.ipynb

In [3]:
setup_conn = ConectionSetup()

Enter User Name:  n1604989
Enter Password:  ········


In [4]:
cnxn = setup_conn.con

In [5]:
ann_contable = 2021
mes_contable = 12
periodo = (ann_contable*100)+ mes_contable
print(periodo)

202112


In [6]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [7]:
inicio = time.time()

aseg_iaxis_query = f"""SELECT DISTINCT PERIODO_CONTABLE 
                            ,SUCURSAL_PROD
                            ,RAMO_PROD
                            ,POLIZA
                            ,CERTIFICADO
                            ,DOCUMENTO as ANEXO
                            ,ANEXO as DOCUMENTO
                            ,NRO_UNICO_ASEG AS SPERSON
                            ,TIPO_IDENTIFI_ASEG
                            ,NRO_IDENTIFI_ASEG
                            ,FI_VIG_ASEG AS FI_CERTIFICADO
                            ,FF_VIG_ASEG AS FF_CERTIFICADO
                            
	FROM PROD.DWH_POL_ASEG_H 
	WHERE COD_PRODUCTO IN (104,105) AND ({periodo}-600)<=PERIODO_CONTABLE
                AND PERIODO_CONTABLE<= {periodo} """

aseg_iaxis_h = pd.read_sql(aseg_iaxis_query,cnxn)

fin = time.time()
3
print("aseg_iaxis_query: " + str((fin-inicio)/float(60)) + " min")

aseg_iaxis_query: 5.700382912158966 min


In [8]:
aseg_iaxis_h.RAMO_PROD = aseg_iaxis_h.RAMO_PROD.astype('string')
aseg_iaxis_h.POLIZA = aseg_iaxis_h.POLIZA.astype('string')
aseg_iaxis_h.CERTIFICADO = aseg_iaxis_h.CERTIFICADO.astype('string')
aseg_iaxis_h.ANEXO = aseg_iaxis_h.ANEXO.astype('string')
aseg_iaxis_h.DOCUMENTO = aseg_iaxis_h.DOCUMENTO.astype('string')
aseg_iaxis_h.FI_CERTIFICADO = aseg_iaxis_h.FI_CERTIFICADO.astype('string')
aseg_iaxis_h.FF_CERTIFICADO = aseg_iaxis_h.FF_CERTIFICADO.astype('string')
aseg_iaxis_h.TIPO_IDENTIFI_ASEG = aseg_iaxis_h.TIPO_IDENTIFI_ASEG.astype('string')
aseg_iaxis_h.NRO_IDENTIFI_ASEG = aseg_iaxis_h.NRO_IDENTIFI_ASEG.astype('string')
aseg_iaxis_h.SUCURSAL_PROD = aseg_iaxis_h.SUCURSAL_PROD.astype('string')
aseg_iaxis_h.PERIODO_CONTABLE = aseg_iaxis_h.PERIODO_CONTABLE.astype('Int64')
aseg_iaxis_h.SPERSON = aseg_iaxis_h.SPERSON.astype('Int64')
aseg_iaxis_h.dtypes

PERIODO_CONTABLE       Int64
SUCURSAL_PROD         string
RAMO_PROD             string
POLIZA                string
CERTIFICADO           string
ANEXO                 string
DOCUMENTO             string
SPERSON                Int64
TIPO_IDENTIFI_ASEG    string
NRO_IDENTIFI_ASEG     string
FI_CERTIFICADO        string
FF_CERTIFICADO        string
dtype: object

In [9]:
#removing spaces 
aseg_iaxis_h.RAMO_PROD = aseg_iaxis_h["RAMO_PROD"].str.strip()
aseg_iaxis_h.POLIZA = aseg_iaxis_h["POLIZA"].str.strip()
aseg_iaxis_h.CERTIFICADO = aseg_iaxis_h["CERTIFICADO"].str.strip()
aseg_iaxis_h.ANEXO = aseg_iaxis_h["ANEXO"].str.strip()
aseg_iaxis_h.DOCUMENTO = aseg_iaxis_h["DOCUMENTO"].str.strip()
aseg_iaxis_h.TIPO_IDENTIFI_ASEG = aseg_iaxis_h["TIPO_IDENTIFI_ASEG"].str.strip()
aseg_iaxis_h.NRO_IDENTIFI_ASEG = aseg_iaxis_h["NRO_IDENTIFI_ASEG"].str.strip()
aseg_iaxis_h.FI_CERTIFICADO = aseg_iaxis_h["FI_CERTIFICADO"].str.strip()
aseg_iaxis_h.FF_CERTIFICADO = aseg_iaxis_h["FF_CERTIFICADO"].str.strip()
aseg_iaxis_h.SUCURSAL_PROD = aseg_iaxis_h["SUCURSAL_PROD"].str.strip()
aseg_iaxis_h.dtypes

PERIODO_CONTABLE       Int64
SUCURSAL_PROD         string
RAMO_PROD             string
POLIZA                string
CERTIFICADO           string
ANEXO                 string
DOCUMENTO             string
SPERSON                Int64
TIPO_IDENTIFI_ASEG    string
NRO_IDENTIFI_ASEG     string
FI_CERTIFICADO        string
FF_CERTIFICADO        string
dtype: object

In [10]:
#deleting commas 
aseg_iaxis_h.FI_CERTIFICADO = aseg_iaxis_h.FI_CERTIFICADO.str[:8]
aseg_iaxis_h.FF_CERTIFICADO = aseg_iaxis_h.FF_CERTIFICADO.str[:8]
aseg_iaxis_h.head()

,PERIODO_CONTABLE,SUCURSAL_PROD,RAMO_PROD,POLIZA,CERTIFICADO,ANEXO,DOCUMENTO,SPERSON,TIPO_IDENTIFI_ASEG,NRO_IDENTIFI_ASEG,FI_CERTIFICADO,FF_CERTIFICADO
0,201608,20.0,6033,168727,0,1,0,331712,C,17095228,20160819,20170819
1,201611,13.0,6046,149272,336,1,0,351120,C,91260486,20161030,20171030
2,201906,27.0,6048,201424,1659867,7,2,2348212,C,1081183054,20190530,20200530
3,202101,226.0,6048,293563,4886569,1,0,23858622,C,1109494135,20200228,20210228
4,202101,226.0,6048,293563,5721948,2,1,24662630,C,4559294,20200812,20210812


In [11]:
#concatenating multiple columns
aseg_iaxis_h["LLAVE_DT"] = aseg_iaxis_h["RAMO_PROD"] + "-" + aseg_iaxis_h["POLIZA"] + "-" + aseg_iaxis_h["CERTIFICADO"] + "-" + aseg_iaxis_h["ANEXO"]
aseg_iaxis_h["LLAVE_V"] = aseg_iaxis_h["RAMO_PROD"] + "-" + aseg_iaxis_h["POLIZA"] + "-" + aseg_iaxis_h["CERTIFICADO"] + "-" + aseg_iaxis_h["FI_CERTIFICADO"]+ "-" + aseg_iaxis_h["FF_CERTIFICADO"]
aseg_iaxis_h.head()

,PERIODO_CONTABLE,SUCURSAL_PROD,RAMO_PROD,POLIZA,CERTIFICADO,ANEXO,DOCUMENTO,SPERSON,TIPO_IDENTIFI_ASEG,NRO_IDENTIFI_ASEG,FI_CERTIFICADO,FF_CERTIFICADO,LLAVE_DT,LLAVE_V
0,201608,20.0,6033,168727,0,1,0,331712,C,17095228,20160819,20170819,6033-168727-0-1,6033-168727-0-20160819-20170819
1,201611,13.0,6046,149272,336,1,0,351120,C,91260486,20161030,20171030,6046-149272-336-1,6046-149272-336-20161030-20171030
2,201906,27.0,6048,201424,1659867,7,2,2348212,C,1081183054,20190530,20200530,6048-201424-1659867-7,6048-201424-1659867-20190530-20200530
3,202101,226.0,6048,293563,4886569,1,0,23858622,C,1109494135,20200228,20210228,6048-293563-4886569-1,6048-293563-4886569-20200228-20210228
4,202101,226.0,6048,293563,5721948,2,1,24662630,C,4559294,20200812,20210812,6048-293563-5721948-2,6048-293563-5721948-20200812-20210812


In [12]:
#casting to datetime
#aseg_iaxis_h.FI_CERTIFICADO = pd.to_datetime(aseg_iaxis_h["FI_CERTIFICADO"], format= '%Y-%m-%d', errors = "coerce")
#aseg_iaxis_h.FF_CERTIFICADO = pd.to_datetime(aseg_iaxis_h["FF_CERTIFICADO"], format= '%Y-%m-%d', errors = "coerce")
#aseg_iaxis_h.head()

In [13]:
inicio1 = time.time()

aseg_as400_query = f"""SELECT DISTINCT PERIODO_CONTABLE
                              ,SUCURSAL_PROD
                              ,RAMO_PROD
                              ,POLIZA
                              ,CERTIFICADO
                              ,DOCUMENTO
                              ,ANEXO
                              /*,. AS SPERSON*/
                              ,TIPO_IDENTIFI_ASEG
                              /*,NRO_IDENTIFI_ASEG AS NRO_ID*/
                              ,FI_VIG_ASEG AS FI_CERTIFICADO
                              ,FF_VIG_ASEG AS FF_CERTIFICADO
              FROM PROD.DWH_POL_ASEG_H
              WHERE ({periodo}-600)<=PERIODO_CONTABLE
              AND PERIODO_CONTABLE<= {periodo}
              AND RAMO_PROD IN ('AL','AT','AU','AW','AX','CT','ET','LM','LO','ME','MO',
												'MT','MU','MW','PE','PM','PO','TO','TP','AWL','TS','TT','TU','MF','LF')
	          ORDER BY RAMO_PROD, POLIZA, CERTIFICADO, DOCUMENTO"""
aseg_as400_h = pd.read_sql(aseg_as400_query,cnxn)

fin1 = time.time()

print("aseg_as400_query: " + str((fin1-inicio1)/float(60)) + " min")

aseg_as400_query: 1.8523198088010153 min


In [14]:
#convert integer to string 
aseg_as400_h.RAMO_PROD = aseg_as400_h.RAMO_PROD.astype('string')
aseg_as400_h.POLIZA = aseg_as400_h.POLIZA.astype('string')
aseg_as400_h.CERTIFICADO = aseg_as400_h.CERTIFICADO.astype('string')
aseg_as400_h.ANEXO = aseg_as400_h.ANEXO.astype('string')
aseg_as400_h.DOCUMENTO = aseg_as400_h.DOCUMENTO.astype('string')
aseg_as400_h.TIPO_IDENTIFI_ASEG = aseg_as400_h.TIPO_IDENTIFI_ASEG.astype('string')
aseg_as400_h.FI_CERTIFICADO = aseg_as400_h.FI_CERTIFICADO.astype('string')
aseg_as400_h.FF_CERTIFICADO = aseg_as400_h.FF_CERTIFICADO.astype('string')
aseg_as400_h.PERIODO_CONTABLE = aseg_as400_h.PERIODO_CONTABLE.astype('Int64')
aseg_as400_h.SUCURSAL_PROD = aseg_as400_h.SUCURSAL_PROD.astype('string')
aseg_as400_h.dtypes

PERIODO_CONTABLE       Int64
SUCURSAL_PROD         string
RAMO_PROD             string
POLIZA                string
CERTIFICADO           string
DOCUMENTO             string
ANEXO                 string
TIPO_IDENTIFI_ASEG    string
FI_CERTIFICADO        string
FF_CERTIFICADO        string
dtype: object

In [15]:
#removing spaces 
aseg_as400_h.RAMO_PROD = aseg_as400_h["RAMO_PROD"].str.strip()
aseg_as400_h.POLIZA = aseg_as400_h["POLIZA"].str.strip()
aseg_as400_h.CERTIFICADO = aseg_as400_h["CERTIFICADO"].str.strip()
aseg_as400_h.ANEXO = aseg_as400_h["CERTIFICADO"].str.strip()
aseg_as400_h.DOCUMENTO = aseg_as400_h["DOCUMENTO"].str.strip()
aseg_as400_h.FI_CERTIFICADO = aseg_as400_h["FI_CERTIFICADO"].str.strip()
aseg_as400_h.FF_CERTIFICADO = aseg_as400_h["FF_CERTIFICADO"].str.strip()
aseg_as400_h.TIPO_IDENTIFI_ASEG = aseg_as400_h["TIPO_IDENTIFI_ASEG"].str.strip()
aseg_as400_h.SUCURSAL_PROD = aseg_as400_h["SUCURSAL_PROD"].str.strip()
aseg_as400_h.head()

,PERIODO_CONTABLE,SUCURSAL_PROD,RAMO_PROD,POLIZA,CERTIFICADO,DOCUMENTO,ANEXO,TIPO_IDENTIFI_ASEG,FI_CERTIFICADO,FF_CERTIFICADO
0,201605,11,AT,8796,114,1201,114,N,20151103,20160426
1,201609,11,AT,8796,149,1001,149,C,20151103,20160531
2,201607,11,AT,8796,161,801,161,C,20151103,20161103
3,201607,11,AT,8796,161,901,161,C,20151103,20161103
4,201607,11,AT,8796,161,901,161,C,20160617,20161103


In [16]:
#deleting commas 
aseg_as400_h.FI_CERTIFICADO = aseg_as400_h.FI_CERTIFICADO.str[:8]
aseg_as400_h.FF_CERTIFICADO = aseg_as400_h.FF_CERTIFICADO.str[:8]
aseg_as400_h.head()

,PERIODO_CONTABLE,SUCURSAL_PROD,RAMO_PROD,POLIZA,CERTIFICADO,DOCUMENTO,ANEXO,TIPO_IDENTIFI_ASEG,FI_CERTIFICADO,FF_CERTIFICADO
0,201605,11,AT,8796,114,1201,114,N,20151103,20160426
1,201609,11,AT,8796,149,1001,149,C,20151103,20160531
2,201607,11,AT,8796,161,801,161,C,20151103,20161103
3,201607,11,AT,8796,161,901,161,C,20151103,20161103
4,201607,11,AT,8796,161,901,161,C,20160617,20161103


In [17]:
aseg_as400_h["LLAVE_DT"] = aseg_as400_h["RAMO_PROD"] + "-" + aseg_as400_h["POLIZA"] + "-" + aseg_as400_h["CERTIFICADO"] + "-" + aseg_as400_h["DOCUMENTO"]
aseg_as400_h["LLAVE_V"] = aseg_as400_h["RAMO_PROD"] + "-" + aseg_as400_h["POLIZA"] + "-" + aseg_as400_h["CERTIFICADO"] + "-" + aseg_as400_h["FI_CERTIFICADO"]+ "-" + aseg_as400_h["FF_CERTIFICADO"]
aseg_as400_h.head()

,PERIODO_CONTABLE,SUCURSAL_PROD,RAMO_PROD,POLIZA,CERTIFICADO,DOCUMENTO,ANEXO,TIPO_IDENTIFI_ASEG,FI_CERTIFICADO,FF_CERTIFICADO,LLAVE_DT,LLAVE_V
0,201605,11,AT,8796,114,1201,114,N,20151103,20160426,AT-8796-114-1201,AT-8796-114-20151103-20160426
1,201609,11,AT,8796,149,1001,149,C,20151103,20160531,AT-8796-149-1001,AT-8796-149-20151103-20160531
2,201607,11,AT,8796,161,801,161,C,20151103,20161103,AT-8796-161-801,AT-8796-161-20151103-20161103
3,201607,11,AT,8796,161,901,161,C,20151103,20161103,AT-8796-161-901,AT-8796-161-20151103-20161103
4,201607,11,AT,8796,161,901,161,C,20160617,20161103,AT-8796-161-901,AT-8796-161-20160617-20161103


In [18]:
#casting to datetime
##aseg_as400_h.FI_CERTIFICADO = pd.to_datetime(aseg_as400_h["FI_CERTIFICADO"], format= '%Y-%m-%d', errors = "coerce")
##aseg_as400_h.FF_CERTIFICADO = pd.to_datetime(aseg_as400_h["FF_CERTIFICADO"], format= '%Y-%m-%d', errors = "coerce")
##aseg_as400_h.head()

In [19]:
aseg_id_h = pd.concat([aseg_iaxis_h,aseg_as400_h])
aseg_id_h.dtypes

PERIODO_CONTABLE       Int64
SUCURSAL_PROD         string
RAMO_PROD             string
POLIZA                string
CERTIFICADO           string
ANEXO                 string
DOCUMENTO             string
SPERSON                Int64
TIPO_IDENTIFI_ASEG    string
NRO_IDENTIFI_ASEG     string
FI_CERTIFICADO        string
FF_CERTIFICADO        string
LLAVE_DT              string
LLAVE_V               string
dtype: object

In [20]:
aseg_id_h = aseg_id_h.sort_values(["LLAVE_DT","ANEXO","DOCUMENTO","PERIODO_CONTABLE","TIPO_IDENTIFI_ASEG","NRO_IDENTIFI_ASEG","FI_CERTIFICADO","FF_CERTIFICADO"],ascending=(True,True,True,True,True,True,True,True))
aseg_id_h.head()

,PERIODO_CONTABLE,SUCURSAL_PROD,RAMO_PROD,POLIZA,CERTIFICADO,ANEXO,DOCUMENTO,SPERSON,TIPO_IDENTIFI_ASEG,NRO_IDENTIFI_ASEG,FI_CERTIFICADO,FF_CERTIFICADO,LLAVE_DT,LLAVE_V
6434604,201605,13.0,6031,100000,0,2,0,565704,C,37514526,20160507,20170507,6031-100000-0-2,6031-100000-0-20160507-20170507
2195239,201605,13.0,6031,100000,0,3,0,565704,C,37514526,20160507,20170507,6031-100000-0-3,6031-100000-0-20160507-20170507
4742980,201605,37.0,6031,100003,0,2,0,564368,C,41691171,20160506,20170506,6031-100003-0-2,6031-100003-0-20160506-20170506
3075392,201701,37.0,6031,100003,0,4,1,564368,C,41691171,20170107,20170506,6031-100003-0-4,6031-100003-0-20170107-20170506
7248220,201705,37.0,6031,100003,0,5,1,564368,C,41691171,20170506,20180506,6031-100003-0-5,6031-100003-0-20170506-20180506


In [21]:
#setting ranking by "LLAVE_DT"
aseg_id_h["RANK"] = aseg_id_h.groupby("LLAVE_DT").cumcount() +1
aseg_id_h.head()

,PERIODO_CONTABLE,SUCURSAL_PROD,RAMO_PROD,POLIZA,CERTIFICADO,ANEXO,DOCUMENTO,SPERSON,TIPO_IDENTIFI_ASEG,NRO_IDENTIFI_ASEG,FI_CERTIFICADO,FF_CERTIFICADO,LLAVE_DT,LLAVE_V,RANK
6434604,201605,13.0,6031,100000,0,2,0,565704,C,37514526,20160507,20170507,6031-100000-0-2,6031-100000-0-20160507-20170507,1
2195239,201605,13.0,6031,100000,0,3,0,565704,C,37514526,20160507,20170507,6031-100000-0-3,6031-100000-0-20160507-20170507,1
4742980,201605,37.0,6031,100003,0,2,0,564368,C,41691171,20160506,20170506,6031-100003-0-2,6031-100003-0-20160506-20170506,1
3075392,201701,37.0,6031,100003,0,4,1,564368,C,41691171,20170107,20170506,6031-100003-0-4,6031-100003-0-20170107-20170506,1
7248220,201705,37.0,6031,100003,0,5,1,564368,C,41691171,20170506,20180506,6031-100003-0-5,6031-100003-0-20170506-20180506,1


In [22]:
aseg_id_h_count = aseg_id_h.groupby("LLAVE_DT").count()
aseg_id_h_count_1 = aseg_id_h_count["RANK"]
aseg_id_h_count_1.head()

LLAVE_DT
6031-100000-0-2    1
6031-100000-0-3    1
6031-100003-0-2    1
6031-100003-0-4    1
6031-100003-0-5    1
Name: RANK, dtype: int64

In [23]:
aseg_id_h = pd.merge(aseg_id_h,aseg_id_h_count_1,on=["LLAVE_DT"],how="left")
aseg_id_h.dtypes

PERIODO_CONTABLE       Int64
SUCURSAL_PROD         string
RAMO_PROD             string
POLIZA                string
CERTIFICADO           string
ANEXO                 string
DOCUMENTO             string
SPERSON                Int64
TIPO_IDENTIFI_ASEG    string
NRO_IDENTIFI_ASEG     string
FI_CERTIFICADO        string
FF_CERTIFICADO        string
LLAVE_DT              string
LLAVE_V               string
RANK_x                 int64
RANK_y                 int64
dtype: object

In [24]:
len(aseg_id_h)

13043725

In [25]:
aseg_id_h = aseg_id_h[aseg_id_h["RANK_x"]==aseg_id_h["RANK_y"]]
len(aseg_id_h)

7919228

In [26]:
aseg_id_h = aseg_id_h.drop(["RANK_x","RANK_y"], axis = 1)
aseg_id_h.dtypes

PERIODO_CONTABLE       Int64
SUCURSAL_PROD         string
RAMO_PROD             string
POLIZA                string
CERTIFICADO           string
ANEXO                 string
DOCUMENTO             string
SPERSON                Int64
TIPO_IDENTIFI_ASEG    string
NRO_IDENTIFI_ASEG     string
FI_CERTIFICADO        string
FF_CERTIFICADO        string
LLAVE_DT              string
LLAVE_V               string
dtype: object

In [27]:
aseg_id_h.to_csv("aseg_dt.csv")

In [28]:
inicio2 = time.time()

autos_iaxis_query = f"""SELECT RAMO_PROD
                        ,POLIZA
                        ,CERTIFICADO
                        ,DOCUMENTO
                        ,ANEXO
                        ,NRO_PLACA
                        ,NRO_CHASIS
                        ,COD_MARCA
                        ,COD_CLASE
                        ,COD_TIPO
                        ,PESO
                        ,ANN_MODELO
                        ,CILINDRAJE
                        ,CAPACI_OCUPANTES
                        ,IND_ACCESORIOS
                        ,VR_ASEG_ACCESORIOS
                        ,VR_ASEG_CASCO
	          FROM DT.DWH_DT_AUT_AUTOS 
	          WHERE COD_PRODUCTO IN (104,105)"""
autos_iaxis = pd.read_sql_query(autos_iaxis_query,cnxn,dtype={"RAMO_PROD":"string"
                                                             ,"POLIZA":"string"
                                                             ,"CERTIFICADO":"string"
                                                             ,"DOCUMENTO":"string"
                                                             ,"ANEXO":"string"
                                                             ,"NRO_PLACA":"string"
                                                             ,"NRO_CHASIS":"string"
                                                             ,"IND_ACCESORIOS":"string"
                                                             ,"COD_MARCA":"Int64"
                                                             ,"COD_CLASE":"Int64"
                                                             ,"COD_TIPO":"Int64"
                                                             ,"PESO":"Int64"
                                                             ,"ANN_MODELO":"Int64"
                                                             ,"CILINDRAJE":"Int64"
                                                             ,"CAPACI_OCUPANTES":"Int64"
                                                             ,"VR_ASEG_ACCESORIOS":"float64"
                                                             ,"VR_ASEG_CASCO":"float64"})

fin2 = time.time()

print("autos_iaxis_query: " + str((fin2-inicio2)/float(60)) + " min")

autos_iaxis_query: 4.370258967081706 min


In [29]:
autos_iaxis.RAMO_PROD = autos_iaxis["RAMO_PROD"].str.strip()
autos_iaxis.POLIZA = autos_iaxis["POLIZA"].str.strip()
autos_iaxis.CERTIFICADO = autos_iaxis["CERTIFICADO"].str.strip()
autos_iaxis.DOCUMENTO = autos_iaxis["DOCUMENTO"].str.strip()
autos_iaxis.ANEXO = autos_iaxis["ANEXO"].str.strip()
autos_iaxis.NRO_PLACA = autos_iaxis["NRO_PLACA"].str.strip()
autos_iaxis.NRO_CHASIS = autos_iaxis["NRO_CHASIS"].str.strip()
autos_iaxis.IND_ACCESORIOS = autos_iaxis["IND_ACCESORIOS"].str.strip()

In [30]:
autos_iaxis["LLAVE_DT"] = autos_iaxis["RAMO_PROD"] + "-" + autos_iaxis["POLIZA"] + "-" + autos_iaxis["CERTIFICADO"] + "-" + autos_iaxis["DOCUMENTO"]
autos_iaxis.head()

,RAMO_PROD,POLIZA,CERTIFICADO,DOCUMENTO,ANEXO,NRO_PLACA,NRO_CHASIS,COD_MARCA,COD_CLASE,COD_TIPO,PESO,ANN_MODELO,CILINDRAJE,CAPACI_OCUPANTES,IND_ACCESORIOS,VR_ASEG_ACCESORIOS,VR_ASEG_CASCO,LLAVE_DT
0,6031,3445,0,1,0,TL7613,JSACHAS1STL7613,16,1,100,0,2014,1400,5,S,150000.0,24700000.0,6031-3445-0-1
1,6031,3445,0,2,1,TL7613,JSACHAS1STL7613,16,1,100,0,2014,1400,5,S,150000.0,24700000.0,6031-3445-0-2
2,6031,3445,0,4,2,TL7613,JSACHAS1STL7613,16,1,100,0,2014,1400,5,S,150000.0,24700000.0,6031-3445-0-4
3,6031,3633,0,1,0,TLI845,JSA1236545,16,1,73,0,2004,1600,5,,0.0,11400000.0,6031-3633-0-1
4,6031,3633,0,2,0,TLI845,JSA1236545,16,1,73,0,2004,1600,5,,0.0,11400000.0,6031-3633-0-2


In [31]:
autos_iaxis = autos_iaxis.sort_values(["LLAVE_DT","NRO_PLACA","DOCUMENTO"], ascending = (True,True,True))
autos_iaxis.head()

,RAMO_PROD,POLIZA,CERTIFICADO,DOCUMENTO,ANEXO,NRO_PLACA,NRO_CHASIS,COD_MARCA,COD_CLASE,COD_TIPO,PESO,ANN_MODELO,CILINDRAJE,CAPACI_OCUPANTES,IND_ACCESORIOS,VR_ASEG_ACCESORIOS,VR_ASEG_CASCO,LLAVE_DT
435637,6031,100000,0,1,0,,,0,0,0,0,<NA>,0,0,S,0.0,40500000.0,6031-100000-0-1
435638,6031,100000,0,2,0,RLR984,3GNAL7EC1BS679545,16,6,213,0,2011,2384,5,S,0.0,44700000.0,6031-100000-0-2
435639,6031,100000,0,3,0,RLR984,3GNAL7EC1BS679545,16,6,213,0,2011,2384,5,S,0.0,44700000.0,6031-100000-0-3
454392,6031,100003,0,1,0,,,0,0,0,0,<NA>,0,0,S,0.0,24100000.0,6031-100003-0-1
454393,6031,100003,0,2,0,CDN702,9FCBK456580102489,56,2,422,0,2008,1600,5,S,0.0,22900000.0,6031-100003-0-2


In [32]:
autos_iaxis["RANK"] = autos_iaxis.groupby("LLAVE_DT").cumcount() +1
autos_iaxis.head()

,RAMO_PROD,POLIZA,CERTIFICADO,DOCUMENTO,ANEXO,NRO_PLACA,NRO_CHASIS,COD_MARCA,COD_CLASE,COD_TIPO,PESO,ANN_MODELO,CILINDRAJE,CAPACI_OCUPANTES,IND_ACCESORIOS,VR_ASEG_ACCESORIOS,VR_ASEG_CASCO,LLAVE_DT,RANK
435637,6031,100000,0,1,0,,,0,0,0,0,<NA>,0,0,S,0.0,40500000.0,6031-100000-0-1,1
435638,6031,100000,0,2,0,RLR984,3GNAL7EC1BS679545,16,6,213,0,2011,2384,5,S,0.0,44700000.0,6031-100000-0-2,1
435639,6031,100000,0,3,0,RLR984,3GNAL7EC1BS679545,16,6,213,0,2011,2384,5,S,0.0,44700000.0,6031-100000-0-3,1
454392,6031,100003,0,1,0,,,0,0,0,0,<NA>,0,0,S,0.0,24100000.0,6031-100003-0-1,1
454393,6031,100003,0,2,0,CDN702,9FCBK456580102489,56,2,422,0,2008,1600,5,S,0.0,22900000.0,6031-100003-0-2,1


In [33]:
autos_iaxis_count = autos_iaxis.groupby("LLAVE_DT").count()
autos_iaxis_count_1 = autos_iaxis_count["RANK"]
autos_iaxis_count_1.head()

LLAVE_DT
6031-100000-0-1    1
6031-100000-0-2    1
6031-100000-0-3    1
6031-100003-0-1    1
6031-100003-0-2    1
Name: RANK, dtype: int64

In [34]:
autos_iaxis = pd.merge(autos_iaxis,autos_iaxis_count_1,on=["LLAVE_DT"],how="left")
len(autos_iaxis)

11948777

In [35]:
autos_iaxis = autos_iaxis[autos_iaxis["RANK_x"]==autos_iaxis["RANK_y"]]
len(autos_iaxis)

4571740

In [36]:
autos_iaxis = autos_iaxis[autos_iaxis.NRO_PLACA != '']
len(autos_iaxis)

4539783

In [37]:
autos_iaxis.loc[autos_iaxis["IND_ACCESORIOS"] == "S", "IND_ACCESORIOS"] = "S"
autos_iaxis.loc[autos_iaxis["IND_ACCESORIOS"] != "S", "IND_ACCESORIOS"] = "N"
len(autos_iaxis)

4539783

In [38]:
autos_iaxis = autos_iaxis.drop(["RANK_x","RANK_y","RAMO_PROD","POLIZA","CERTIFICADO","ANEXO","DOCUMENTO"], axis = 1)
autos_iaxis.head()

,NRO_PLACA,NRO_CHASIS,COD_MARCA,COD_CLASE,COD_TIPO,PESO,ANN_MODELO,CILINDRAJE,CAPACI_OCUPANTES,IND_ACCESORIOS,VR_ASEG_ACCESORIOS,VR_ASEG_CASCO,LLAVE_DT
1,RLR984,3GNAL7EC1BS679545,16,6,213,0,2011,2384,5,S,0.0,44700000.0,6031-100000-0-2
2,RLR984,3GNAL7EC1BS679545,16,6,213,0,2011,2384,5,S,0.0,44700000.0,6031-100000-0-3
4,CDN702,9FCBK456580102489,56,2,422,0,2008,1600,5,S,0.0,22900000.0,6031-100003-0-2
5,UUL857,3N1CN7AD5ZK151997,64,1,218,0,2016,1598,5,S,200000.0,42100000.0,6031-100003-0-4
6,UUL857,3N1CN7AD5ZK151997,64,1,218,0,2016,1598,5,S,200000.0,42100000.0,6031-100003-0-5


In [39]:
inicio3 = time.time()

polizas_iaxis_query = f"""SELECT PERIODO_CONTABLE
                          ,SSEGURO
                          ,SUCURSAL_PROD
                          ,RAMO_PROD
                          ,POLIZA
                          ,CERTIFICADO
                          ,DOCUMENTO
                          ,ANEXO
                          ,RECIBO
                          ,FI_POLIZA_MATRIZ
                          ,FI_CERTIFICADO
                          ,FF_CERTIFICADO
                          ,FI_ANEXO
                          ,FF_ANEXO
                          ,FI_DOCUMENTO
                          ,FF_DOCUMENTO
                          ,FI_VIGENCIA_ANT
                          ,FECHA_EXPE
                          ,TIPO_TRANSAC
                          ,FREC_PAGO
                          ,NRO_UNICO_TOMADOR
                          ,TIPO_IDENTIFI_TOM
                          ,NRO_IDENTIFI_TOM
                          ,INTERMEDIARIO_LIDE
                          ,CANAL
                          ,SEGMENTO
                          ,VR_ASEG_POLIZA
                          ,VR_PRIMA_MN_ORIG
                          ,VR_PRIMA_PESOS
                          ,VR_PRIMA_ANUAL
                          ,COD_DEPARTAMENTO
                          ,COD_CIUDAD
                          ,LLAVE
                          ,'IAXIS' AS ORIGEN
	      FROM PROD.DWH_POLIZAS_H 
	      WHERE ({periodo}-600)<=PERIODO_CONTABLE
          AND PERIODO_CONTABLE<= {periodo}
          /*ANNO_CONTABLE=&ANO AND MES_CONTABLE=&MES*/ 
          AND COD_PRODUCTO IN (104,105)"""
polizas_iaxis_h = pd.read_sql_query(polizas_iaxis_query,cnxn,dtype={"PERIODO_CONTABLE":"Int64","SSEGURO":"Int64","SUCURSAL_PROD":"string","RAMO_PROD":"string","POLIZA":"string","CERTIFICADO":"string","DOCUMENTO":"string","ANEXO":"string","RECIBO":"Int64","FI_POLIZA_MATRIZ":"string","FI_CERTIFICADO":"string","FF_CERTIFICADO":"string","FI_ANEXO":"string","FF_ANEXO":"string","FI_DOCUMENTO":"string","FF_DOCUMENTO":"string","FI_VIGENCIA_ANT":"string","FECHA_EXPE":"string","TIPO_TRANSAC":"Int64","FREC_PAGO":"Int64","NRO_UNICO_TOMADOR":"Int64","TIPO_IDENTIFI_TOM":"string","NRO_IDENTIFI_TOM":"string","INTERMEDIARIO_LIDE":"Int64","CANAL":"string","SEGMENTO":"Int64","VR_ASEG_POLIZA":"float64","VR_PRIMA_MN_ORIG":"float64","VR_PRIMA_PESOS":"float64","VR_PRIMA_ANUAL":"float64","COD_DEPARTAMENTO":"string","COD_CIUDAD":"string","LLAVE":"string","ORIGEN":"string"})

fin3 = time.time()

print("polizas_iaxis_query: " + str((fin3-inicio3)/float(60)) + " min")

polizas_iaxis_query: 8.285781697432201 min


In [40]:
polizas_iaxis_h.RAMO_PROD = polizas_iaxis_h["RAMO_PROD"].str.strip()
polizas_iaxis_h.POLIZA = polizas_iaxis_h["POLIZA"].str.strip()
polizas_iaxis_h.CERTIFICADO = polizas_iaxis_h["CERTIFICADO"].str.strip()
polizas_iaxis_h.DOCUMENTO = polizas_iaxis_h["DOCUMENTO"].str.strip()
polizas_iaxis_h.ANEXO = polizas_iaxis_h["ANEXO"].str.strip()
polizas_iaxis_h.FI_CERTIFICADO = polizas_iaxis_h["FI_CERTIFICADO"].str.strip()
polizas_iaxis_h.FF_CERTIFICADO = polizas_iaxis_h["FF_CERTIFICADO"].str.strip()
polizas_iaxis_h.SUCURSAL_PROD = polizas_iaxis_h["SUCURSAL_PROD"].str.strip()
polizas_iaxis_h.TIPO_IDENTIFI_TOM = polizas_iaxis_h["TIPO_IDENTIFI_TOM"].str.strip()
polizas_iaxis_h.NRO_IDENTIFI_TOM = polizas_iaxis_h["NRO_IDENTIFI_TOM"].str.strip()
polizas_iaxis_h.CANAL = polizas_iaxis_h["CANAL"].str.strip()
polizas_iaxis_h.COD_DEPARTAMENTO = polizas_iaxis_h["COD_DEPARTAMENTO"].str.strip()
polizas_iaxis_h.COD_CIUDAD = polizas_iaxis_h["COD_CIUDAD"].str.strip()
polizas_iaxis_h.LLAVE = polizas_iaxis_h["LLAVE"].str.strip()

In [42]:
polizas_iaxis_h.FI_CERTIFICADO = polizas_iaxis_h.FI_CERTIFICADO.str[:8]
polizas_iaxis_h.FF_CERTIFICADO = polizas_iaxis_h.FF_CERTIFICADO.str[:8]

In [43]:
polizas_iaxis_h["LLAVE_DT"] = polizas_iaxis_h["RAMO_PROD"] + "-" + polizas_iaxis_h["POLIZA"] + "-" + polizas_iaxis_h["CERTIFICADO"] + "-" + polizas_iaxis_h["ANEXO"]
polizas_iaxis_h["LLAVE_V"] = polizas_iaxis_h["RAMO_PROD"] + "-" + polizas_iaxis_h["POLIZA"] + "-" + polizas_iaxis_h["CERTIFICADO"] + "-" + polizas_iaxis_h["FI_CERTIFICADO"]+ "-" + polizas_iaxis_h["FF_CERTIFICADO"]

In [ ]:
'''polizas_iaxis_h["FI_CERTIFICADO"] = pd.to_datetime(polizas_iaxis_h["FI_CERTIFICADO"], format= '%Y-%m-%d', errors = "coerce")
polizas_iaxis_h["FF_CERTIFICADO"] = pd.to_datetime(polizas_iaxis_h["FF_CERTIFICADO"], format= '%Y-%m-%d', errors = "coerce")
polizas_iaxis_h["FI_ANEXO"] = pd.to_datetime(polizas_iaxis_h["FI_ANEXO"], format= '%Y-%m-%d', errors = "coerce")
polizas_iaxis_h["FF_ANEXO"] = pd.to_datetime(polizas_iaxis_h["FF_ANEXO"], format= '%Y-%m-%d', errors = "coerce")
polizas_iaxis_h["FI_DOCUMENTO"] = pd.to_datetime(polizas_iaxis_h["FI_DOCUMENTO"], format= '%Y-%m-%d', errors = "coerce")
polizas_iaxis_h["FF_DOCUMENTO"] = pd.to_datetime(polizas_iaxis_h["FF_DOCUMENTO"], format= '%Y-%m-%d', errors = "coerce")
polizas_iaxis_h["FI_POLIZA_MATRIZ"] = pd.to_datetime(polizas_iaxis_h["FI_POLIZA_MATRIZ"], format= '%Y-%m-%d', errors = "coerce")
polizas_iaxis_h["FI_VIGENCIA_ANT"] = pd.to_datetime(polizas_iaxis_h["FI_VIGENCIA_ANT"], format= '%Y-%m-%d', errors = "coerce")
polizas_iaxis_h["FECHA_EXPE"] = pd.to_datetime(polizas_iaxis_h["FECHA_EXPE"], format= '%Y-%m-%d', errors = "coerce")
polizas_iaxis_h.dtypes'''

In [44]:
polizas_iaxis_h = polizas_iaxis_h.sort_values("LLAVE_DT", ascending = True)
len(polizas_iaxis_h)

9752647

In [45]:
inicio4 = time.time()

pol_aux_query = f"""SELECT DISTINCT LLAVE
                              /*,PERIODO_CONTABLE
                              ,SUCURSAL_PROD
                              ,RAMO_PROD
                              ,COD_PRODUCTO
                              ,POLIZA
                              ,CERTIFICADO
                              ,DOCUMENTO AS ANEXO
                              ,ANEXO AS DOCUMENTO
                              ,FI_CERTIFICADO
                              ,FF_CERTIFICADO*/
                              ,COD_MODALIDAD AS TIPO_RIESGO
               FROM PROD.DWH_POLIZAS_H
	WHERE RAMO_PROD = '900753'"""
pol_aux = pd.read_sql_query(pol_aux_query,cnxn,dtype={"LLAVE":"string","TIPO_RIESGO":"Int64"})

fin4 = time.time()

print("pol_aux_query: " + str((fin4-inicio4)/float(60)) + " sec")

pol_aux_query: 1.4037570794423422 sec


In [46]:
pol_aux.LLAVE = pol_aux["LLAVE"].str.strip()
pol_aux.head()

,LLAVE,TIPO_RIESGO
0,0011535003000146036836,1
1,00179454890001128654829,1
2,00168286700001130599554,1
3,00258934440001170918310,1
4,00268326260001173495889,1


In [47]:
polizas_autos_iaxis = pd.merge(polizas_iaxis_h,autos_iaxis,on=["LLAVE_DT"],how="left")
polizas_autos_iaxis.dtypes

PERIODO_CONTABLE        Int64
SSEGURO                 Int64
SUCURSAL_PROD          string
RAMO_PROD              string
POLIZA                 string
CERTIFICADO            string
DOCUMENTO              string
ANEXO                  string
RECIBO                  Int64
FI_POLIZA_MATRIZ       string
FI_CERTIFICADO         string
FF_CERTIFICADO         string
FI_ANEXO               string
FF_ANEXO               string
FI_DOCUMENTO           string
FF_DOCUMENTO           string
FI_VIGENCIA_ANT        string
FECHA_EXPE             string
TIPO_TRANSAC            Int64
FREC_PAGO               Int64
NRO_UNICO_TOMADOR       Int64
TIPO_IDENTIFI_TOM      string
NRO_IDENTIFI_TOM       string
INTERMEDIARIO_LIDE      Int64
CANAL                  string
SEGMENTO                Int64
VR_ASEG_POLIZA        float64
VR_PRIMA_MN_ORIG      float64
VR_PRIMA_PESOS        float64
VR_PRIMA_ANUAL        float64
COD_DEPARTAMENTO       string
COD_CIUDAD             string
LLAVE                  string
ORIGEN    

In [48]:
base_1 = pd.merge(polizas_autos_iaxis,pol_aux,on=["LLAVE"],how="left")
base_1.head()

,PERIODO_CONTABLE,SSEGURO,SUCURSAL_PROD,RAMO_PROD,POLIZA,CERTIFICADO,DOCUMENTO,ANEXO,RECIBO,FI_POLIZA_MATRIZ,FI_CERTIFICADO,FF_CERTIFICADO,FI_ANEXO,FF_ANEXO,FI_DOCUMENTO,FF_DOCUMENTO,FI_VIGENCIA_ANT,FECHA_EXPE,TIPO_TRANSAC,FREC_PAGO,NRO_UNICO_TOMADOR,TIPO_IDENTIFI_TOM,NRO_IDENTIFI_TOM,INTERMEDIARIO_LIDE,CANAL,SEGMENTO,VR_ASEG_POLIZA,VR_PRIMA_MN_ORIG,VR_PRIMA_PESOS,VR_PRIMA_ANUAL,COD_DEPARTAMENTO,COD_CIUDAD,LLAVE,ORIGEN,LLAVE_DT,LLAVE_V,NRO_PLACA,NRO_CHASIS,COD_MARCA,COD_CLASE,COD_TIPO,PESO,ANN_MODELO,CILINDRAJE,CAPACI_OCUPANTES,IND_ACCESORIOS,VR_ASEG_ACCESORIOS,VR_ASEG_CASCO,TIPO_RIESGO
0,201605,857251,13,6031,100000,0,2,0,2554383,20150507,20160507,20170507,20160507.0,20170507.0,20160507,20170507.0,20150507,20160502,2,5,565704,C,37514526,13336,1,1,3.042188e+09,1146967.0,1146967.0,1146966.96,68,1,0000857251000208002425,IAXIS,6031-100000-0-0,6031-100000-0-20160507-20170507,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>
1,201605,857251,13,6031,100000,0,3,0,2627626,20150507,20160507,20170507,20160507.0,20170507.0,20160507,20170507.0,20150507,20160516,4,5,565704,C,37514526,13336,1,1,3.042188e+09,-1146967.0,-1146967.0,1146966.96,68,1,0000857251000308206344,IAXIS,6031-100000-0-0,6031-100000-0-20160507-20170507,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>
2,201605,839600,37,6031,100003,0,2,0,2466007,20150506,20160506,20170506,20160506.0,20170506.0,20160506,20170506.0,20150506,20160422,2,5,564368,C,41691171,37166,1,1,3.071188e+09,763230.0,763230.0,763229.55,11,1,0000839600000207785135,IAXIS,6031-100003-0-0,6031-100003-0-20160506-20170506,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>
3,201804,839600,37,6031,100003,0,6,1,9473473,20150506,20180506,20190506,20180506.0,20190506.0,20180506,20190506.0,20170506,20180420,2,5,564368,C,41691171,37166,1,1,3.111456e+09,1302616.0,1302616.0,1302617.50,11,1,0000839600000623985960,IAXIS,6031-100003-0-1,6031-100003-0-20180506-20190506,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>
4,201705,839600,37,6031,100003,0,5,1,4908816,20150506,20170506,20180506,20170506.0,20180506.0,20170506,20180506.0,20160506,20170417,2,5,564368,C,41691171,37166,1,1,3.072433e+09,1188728.0,1188728.0,1188729.37,11,1,0000839600000514516920,IAXIS,6031-100003-0-1,6031-100003-0-20170506-20180506,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>


In [ ]:
base_1 = base_1.drop(["LLAVE"], axis = 1)

In [50]:
base_1 = base_1.sort_values(["LLAVE_V","LLAVE_DT"], ascending = (True,True))
base_1.dtypes

PERIODO_CONTABLE        Int64
SSEGURO                 Int64
SUCURSAL_PROD          string
RAMO_PROD              string
POLIZA                 string
CERTIFICADO            string
DOCUMENTO              string
ANEXO                  string
RECIBO                  Int64
FI_POLIZA_MATRIZ       string
FI_CERTIFICADO         string
FF_CERTIFICADO         string
FI_ANEXO               string
FF_ANEXO               string
FI_DOCUMENTO           string
FF_DOCUMENTO           string
FI_VIGENCIA_ANT        string
FECHA_EXPE             string
TIPO_TRANSAC            Int64
FREC_PAGO               Int64
NRO_UNICO_TOMADOR       Int64
TIPO_IDENTIFI_TOM      string
NRO_IDENTIFI_TOM       string
INTERMEDIARIO_LIDE      Int64
CANAL                  string
SEGMENTO                Int64
VR_ASEG_POLIZA        float64
VR_PRIMA_MN_ORIG      float64
VR_PRIMA_PESOS        float64
VR_PRIMA_ANUAL        float64
COD_DEPARTAMENTO       string
COD_CIUDAD             string
ORIGEN                 string
LLAVE_DT  

In [51]:
inicio5 = time.time()

autos_as400_query = f"""SELECT DISTINCT RAMO_PROD
                              ,POLIZA
                              ,CERTIFICADO
                              ,DOCUMENTO
                              ,ANEXO
                              ,NRO_PLACA
                              ,NRO_CHASIS
                              ,COD_MARCA
                              ,COD_CLASE
                              ,COD_TIPO
                              ,PESO
                              ,ANN_MODELO
                              ,CILINDRAJE
                              ,CAPACI_OCUPANTES
                              ,IND_ACCESORIOS
                              ,VR_ASEG_ACCESORIOS
                              ,VR_ASEG_CASCO
                              /*,(ANN_CONTABLE*100+MES_CONTABLE) AS PERIODO_CONTABLE*/
	            FROM DT.DWH_DT_AUT_AUTOS 
                WHERE COD_PRODUCTO IN (104,105) 
                AND ({periodo}-600)<=(ANN_CONTABLE*100+MES_CONTABLE)
                AND (ANN_CONTABLE*100+MES_CONTABLE)<= {periodo}
                AND RAMO_PROD <> 'AO'"""
autos_as400 = pd.read_sql_query(autos_as400_query,cnxn,dtype={"RAMO_PROD":"string","POLIZA":"string","CERTIFICADO":"string","DOCUMENTO":"string","ANEXO":"string","NRO_PLACA":"string","NRO_CHASIS":"string","IND_ACCESORIOS":"string","COD_MARCA":"Int64","COD_CLASE":"Int64","COD_TIPO":"Int64","PESO":"Int64","ANN_MODELO":"Int64","CILINDRAJE":"Int64","CAPACI_OCUPANTES":"Int64","VR_ASEG_ACCESORIOS":"float64","VR_ASEG_CASCO":"float64"})

fin5 = time.time()

print("autos_as400_query: " + str((fin5-inicio5)/float(60)) + " min")

autos_as400_query: 1.8551533182462057 min


In [52]:
autos_as400.RAMO_PROD = autos_as400["RAMO_PROD"].str.strip()
autos_as400.POLIZA = autos_as400["POLIZA"].str.strip()
autos_as400.CERTIFICADO = autos_as400["CERTIFICADO"].str.strip()
autos_as400.DOCUMENTO = autos_as400["DOCUMENTO"].str.strip()
autos_as400.ANEXO = autos_as400["ANEXO"].str.strip()
autos_as400.NRO_PLACA = autos_as400["NRO_PLACA"].str.strip()
autos_as400.NRO_CHASIS = autos_as400["NRO_CHASIS"].str.strip()
autos_as400.IND_ACCESORIOS = autos_as400["IND_ACCESORIOS"].str.strip()
autos_as400.NRO_PLACA = autos_as400["NRO_PLACA"].str.strip()

In [53]:
autos_as400["LLAVE_DT"] = autos_as400["RAMO_PROD"] + "-" + autos_as400["POLIZA"] + "-" + autos_as400["CERTIFICADO"] + "-" + autos_as400["DOCUMENTO"]

In [54]:
autos_as400 = autos_as400.sort_values(["LLAVE_DT","NRO_PLACA","DOCUMENTO"], ascending = (True,True,True))
autos_as400.head(3)

,RAMO_PROD,POLIZA,CERTIFICADO,DOCUMENTO,ANEXO,NRO_PLACA,NRO_CHASIS,COD_MARCA,COD_CLASE,COD_TIPO,PESO,ANN_MODELO,CILINDRAJE,CAPACI_OCUPANTES,IND_ACCESORIOS,VR_ASEG_ACCESORIOS,VR_ASEG_CASCO,LLAVE_DT
163597,6031,100000,0,2,0,RLR984,3GNAL7EC1BS679545,16,6,213,0,2011,2384,5,S,0.0,44700000.0,6031-100000-0-2
172174,6031,100000,0,3,0,RLR984,3GNAL7EC1BS679545,16,6,213,0,2011,2384,5,S,0.0,44700000.0,6031-100000-0-3
362754,6031,100003,0,2,0,CDN702,9FCBK456580102489,56,2,422,0,2008,1600,5,S,0.0,22900000.0,6031-100003-0-2


In [55]:
autos_as400["RANK"] = autos_as400.groupby("LLAVE_DT").cumcount() +1

In [56]:
autos_as400_count = autos_as400.groupby("LLAVE_DT").count()
autos_as400_count_1 = autos_as400_count["RANK"]
autos_as400_count_1.head()

LLAVE_DT
6031-100000-0-2    1
6031-100000-0-3    1
6031-100003-0-2    1
6031-100003-0-4    1
6031-100003-0-5    1
Name: RANK, dtype: int64

In [57]:
autos_as400 = pd.merge(autos_as400,autos_as400_count_1,on=["LLAVE_DT"],how="left")
len(autos_as400)

3645210

In [58]:
autos_as400 = autos_as400[autos_as400["RANK_x"]==autos_as400["RANK_y"]]
len(autos_as400)

3637652

In [59]:
autos_as400 = autos_as400[autos_as400.NRO_PLACA != '']
len(autos_as400)

3613270

In [60]:
autos_as400.loc[autos_as400["IND_ACCESORIOS"] == "S", "IND_ACCESORIOS"] = "S"
autos_as400.loc[autos_as400["IND_ACCESORIOS"] != "S", "IND_ACCESORIOS"] = "N"
autos_as400.head(3)

,RAMO_PROD,POLIZA,CERTIFICADO,DOCUMENTO,ANEXO,NRO_PLACA,NRO_CHASIS,COD_MARCA,COD_CLASE,COD_TIPO,PESO,ANN_MODELO,CILINDRAJE,CAPACI_OCUPANTES,IND_ACCESORIOS,VR_ASEG_ACCESORIOS,VR_ASEG_CASCO,LLAVE_DT,RANK_x,RANK_y
0,6031,100000,0,2,0,RLR984,3GNAL7EC1BS679545,16,6,213,0,2011,2384,5,S,0.0,44700000.0,6031-100000-0-2,1,1
1,6031,100000,0,3,0,RLR984,3GNAL7EC1BS679545,16,6,213,0,2011,2384,5,S,0.0,44700000.0,6031-100000-0-3,1,1
2,6031,100003,0,2,0,CDN702,9FCBK456580102489,56,2,422,0,2008,1600,5,S,0.0,22900000.0,6031-100003-0-2,1,1


In [61]:
autos_as400 = autos_as400.drop(["RANK_x","RANK_y","RAMO_PROD","POLIZA","CERTIFICADO","ANEXO","DOCUMENTO"], axis = 1)
autos_as400.head()

,NRO_PLACA,NRO_CHASIS,COD_MARCA,COD_CLASE,COD_TIPO,PESO,ANN_MODELO,CILINDRAJE,CAPACI_OCUPANTES,IND_ACCESORIOS,VR_ASEG_ACCESORIOS,VR_ASEG_CASCO,LLAVE_DT
0,RLR984,3GNAL7EC1BS679545,16,6,213,0,2011,2384,5,S,0.0,44700000.0,6031-100000-0-2
1,RLR984,3GNAL7EC1BS679545,16,6,213,0,2011,2384,5,S,0.0,44700000.0,6031-100000-0-3
2,CDN702,9FCBK456580102489,56,2,422,0,2008,1600,5,S,0.0,22900000.0,6031-100003-0-2
3,UUL857,3N1CN7AD5ZK151997,64,1,218,0,2016,1598,5,S,200000.0,42100000.0,6031-100003-0-4
4,UUL857,3N1CN7AD5ZK151997,64,1,218,0,2016,1598,5,S,200000.0,42100000.0,6031-100003-0-5


In [ ]:
#cambiar a formato COMMA20. VR_ASEG_ACCESORIOS, VR_ASEG_CASO

In [62]:
inicio6 = time.time()

polizas_as400_query = f"""SELECT PERIODO_CONTABLE 
                     ,SUCURSAL_PROD
                     ,RAMO_PROD
                     ,POLIZA
                     ,CERTIFICADO
                     ,DOCUMENTO
                     ,ANEXO
                     ,FI_POLIZA_MATRIZ
                     ,FI_CERTIFICADO
                     ,FF_CERTIFICADO
                     ,FI_ANEXO
                     ,FF_ANEXO
                     ,FI_DOCUMENTO
                     ,FF_DOCUMENTO
                     ,FI_VIGENCIA_ANT
                     ,FECHA_EXPE
                     ,TIPO_TRANSAC
                     ,FREC_PAGO
                     ,TIPO_IDENTIFI_TOM
                     ,NRO_IDENTIFI_TOM
                     ,INTERMEDIARIO_LIDE
                     ,CANAL
                     ,VR_ASEG_POLIZA
                     ,VR_PRIMA_MN_ORIG
                     ,VR_PRIMA_PESOS
                     ,VR_PRIMA_ANUAL
                     ,COD_DEPARTAMENTO
                     ,COD_CIUDAD
                     ,'AS400' AS ORIGEN
	      FROM PROD.DWH_POLIZAS_H 
	      WHERE ({periodo}-600)<=PERIODO_CONTABLE
          AND PERIODO_CONTABLE<= {periodo}
          /*ANNO_CONTABLE=&ANO AND MES_CONTABLE=&MES*/ 
          AND RAMO_PROD IN ('AL','AT','AU','AW','AX','CT','ET','LM','LO','ME','MO','MT','MU','MW','PE','PM','PO','TO','TP','AWL','TS','TT','TU','MF','LF'/*,'900753'*/)"""
polizas_as400_h = pd.read_sql_query(polizas_as400_query,cnxn,dtype={"PERIODO_CONTABLE":"Int64"
                                                                   ,"SUCURSAL_PROD":"string"
                                                                   ,"RAMO_PROD":"string"
                                                                   ,"POLIZA":"string"
                                                                   ,"CERTIFICADO":"string"
                                                                   ,"DOCUMENTO":"string"
                                                                   ,"ANEXO":"string"
                                                                   ,"FI_POLIZA_MATRIZ":"string"
                                                                   ,"FI_CERTIFICADO":"string"
                                                                   ,"FF_CERTIFICADO":"string"
                                                                   ,"FI_ANEXO":"string"
                                                                   ,"FF_ANEXO":"string"
                                                                   ,"FI_DOCUMENTO":"string"
                                                                   ,"FF_DOCUMENTO":"string"
                                                                   ,"FI_VIGENCIA_ANT":"string"
                                                                   ,"FECHA_EXPE":"string"
                                                                   ,"TIPO_TRANSAC":"Int64"
                                                                   ,"FREC_PAGO":"Int64"
                                                                   ,"TIPO_IDENTIFI_TOM":"string"
                                                                   ,"NRO_IDENTIFI_TOM":"string"
                                                                   ,"INTERMEDIARIO_LIDE":"Int64"
                                                                   ,"CANAL":"string"
                                                                   ,"VR_ASEG_POLIZA":"float64"
                                                                   ,"VR_PRIMA_MN_ORIG":"float64"
                                                                   ,"VR_PRIMA_PESOS":"float64"
                                                                   ,"VR_PRIMA_ANUAL":"float64"
                                                                   ,"COD_DEPARTAMENTO":"string"
                                                                   ,"COD_CIUDAD":"string"
                                                                   ,"ORIGEN":"string"})

fin6 = time.time()

print("polizas_as400_query: " + str((fin6-inicio6)/float(60)) + " min")

polizas_as400_query: 3.2565172791481016 min


In [63]:
polizas_as400_h.RAMO_PROD = polizas_as400_h["RAMO_PROD"].str.strip()
polizas_as400_h.POLIZA = polizas_as400_h["POLIZA"].str.strip()
polizas_as400_h.DOCUMENTO = polizas_as400_h["DOCUMENTO"].str.strip()
polizas_as400_h.CERTIFICADO = polizas_as400_h["CERTIFICADO"].str.strip()
polizas_as400_h.ANEXO = polizas_as400_h["ANEXO"].str.strip()
polizas_as400_h.FI_CERTIFICADO = polizas_as400_h["FI_CERTIFICADO"].str.strip()
polizas_as400_h.FF_CERTIFICADO = polizas_as400_h["FF_CERTIFICADO"].str.strip()
polizas_as400_h.NRO_IDENTIFI_TOM = polizas_as400_h["NRO_IDENTIFI_TOM"].str.strip()
polizas_as400_h.SUCURSAL_PROD = polizas_as400_h["SUCURSAL_PROD"].str.strip()
polizas_as400_h.TIPO_IDENTIFI_TOM = polizas_as400_h["TIPO_IDENTIFI_TOM"].str.strip()
polizas_as400_h.CANAL = polizas_as400_h["CANAL"].str.strip()
polizas_as400_h.COD_CIUDAD = polizas_as400_h["COD_CIUDAD"].str.strip()
polizas_as400_h.COD_DEPARTAMENTO = polizas_as400_h["COD_DEPARTAMENTO"].str.strip()

In [64]:
polizas_as400_h.FI_CERTIFICADO = polizas_as400_h.FI_CERTIFICADO.str[:8]
polizas_as400_h.FF_CERTIFICADO = polizas_as400_h.FF_CERTIFICADO.str[:8]

In [65]:
polizas_as400_h["LLAVE_DT"] = polizas_as400_h["RAMO_PROD"] + "-" + polizas_as400_h["POLIZA"] + "-" + polizas_as400_h["CERTIFICADO"] + "-" + polizas_as400_h["DOCUMENTO"]
polizas_as400_h["LLAVE_V"] = polizas_as400_h["RAMO_PROD"] + "-" + polizas_as400_h["POLIZA"] + "-" + polizas_as400_h["CERTIFICADO"] + "-" + polizas_as400_h["FI_CERTIFICADO"]+ "-" + polizas_as400_h["FF_CERTIFICADO"]

In [ ]:
'''polizas_as400_h.FI_CERTIFICADO = pd.to_datetime(polizas_as400_h["FI_CERTIFICADO"], format= '%Y-%m-%d', errors = "coerce")
polizas_as400_h.FF_CERTIFICADO = pd.to_datetime(polizas_as400_h["FF_CERTIFICADO"], format= '%Y-%m-%d', errors = "coerce")
polizas_as400_h.FI_ANEXO = pd.to_datetime(polizas_as400_h["FI_ANEXO"], format= '%Y-%m-%d', errors = "coerce")
polizas_as400_h.FF_ANEXO = pd.to_datetime(polizas_as400_h["FF_ANEXO"], format= '%Y-%m-%d', errors = "coerce")
polizas_as400_h.FI_DOCUMENTO = pd.to_datetime(polizas_as400_h["FI_DOCUMENTO"], format= '%Y-%m-%d', errors = "coerce")
polizas_as400_h.FF_DOCUMENTO = pd.to_datetime(polizas_as400_h["FF_DOCUMENTO"], format= '%Y-%m-%d', errors = "coerce")
polizas_as400_h.FI_POLIZA_MATRIZ = pd.to_datetime(polizas_as400_h["FI_POLIZA_MATRIZ"], format= '%Y-%m-%d', errors = "coerce")
polizas_as400_h.FI_VIGENCIA_ANT = pd.to_datetime(polizas_as400_h["FI_VIGENCIA_ANT"], format= '%Y-%m-%d', errors = "coerce")
polizas_as400_h.FECHA_EXPE = pd.to_datetime(polizas_as400_h["FECHA_EXPE"], format= '%Y-%m-%d', errors = "coerce")
polizas_as400_h.dtypes'''

In [66]:
base_2 = pd.merge(polizas_as400_h,autos_as400,on=["LLAVE_DT"],how="left")
base_2.head()

,PERIODO_CONTABLE,SUCURSAL_PROD,RAMO_PROD,POLIZA,CERTIFICADO,DOCUMENTO,ANEXO,FI_POLIZA_MATRIZ,FI_CERTIFICADO,FF_CERTIFICADO,FI_ANEXO,FF_ANEXO,FI_DOCUMENTO,FF_DOCUMENTO,FI_VIGENCIA_ANT,FECHA_EXPE,TIPO_TRANSAC,FREC_PAGO,TIPO_IDENTIFI_TOM,NRO_IDENTIFI_TOM,INTERMEDIARIO_LIDE,CANAL,VR_ASEG_POLIZA,VR_PRIMA_MN_ORIG,VR_PRIMA_PESOS,VR_PRIMA_ANUAL,COD_DEPARTAMENTO,COD_CIUDAD,ORIGEN,LLAVE_DT,LLAVE_V,NRO_PLACA,NRO_CHASIS,COD_MARCA,COD_CLASE,COD_TIPO,PESO,ANN_MODELO,CILINDRAJE,CAPACI_OCUPANTES,IND_ACCESORIOS,VR_ASEG_ACCESORIOS,VR_ASEG_CASCO
0,201602,2,AW,7196,106,805,8,20090715,20150901,20160901,20160201,20160201,20160101,20160201,20140901,20160201,9,1,N,800037133,10048,A,3.031216e+09,64929.0,64929.0,779140.0,11,1,AS400,AW-7196-106-805,AW-7196-106-20150901-20160901,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
1,201602,2,AW,7196,114,805,8,20120101,20150901,20160901,20160201,20160201,20160101,20160201,20140901,20160201,9,1,N,800037133,10048,A,3.057121e+09,85917.0,85917.0,1030997.0,11,1,AS400,AW-7196-114-805,AW-7196-114-20150901-20160901,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
2,201602,2,AW,7196,118,405,4,20120101,20150901,20160901,20160201,20160201,20160101,20160201,20140901,20160201,9,1,N,800037133,10048,A,3.023821e+09,53049.0,53049.0,636581.0,11,1,AS400,AW-7196-118-405,AW-7196-118-20150901-20160901,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
3,201602,2,AW,7196,119,405,4,20120101,20150901,20160901,20160201,20160201,20160101,20160201,20140901,20160201,9,1,N,800037133,10048,A,3.028821e+09,52665.0,52665.0,631972.0,11,1,AS400,AW-7196-119-405,AW-7196-119-20150901-20160901,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
4,201602,2,AW,7196,120,605,6,20120101,20150901,20160901,20160201,20160201,20160101,20160201,20140901,20160201,9,1,N,800037133,10048,A,3.029721e+09,62160.0,62160.0,745911.0,11,1,AS400,AW-7196-120-605,AW-7196-120-20150901-20160901,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN


In [67]:
base_pp_mes = pd.concat([base_1,base_2],ignore_index=True)
base_pp_mes.dtypes

PERIODO_CONTABLE        Int64
SSEGURO                 Int64
SUCURSAL_PROD          string
RAMO_PROD              string
POLIZA                 string
CERTIFICADO            string
DOCUMENTO              string
ANEXO                  string
RECIBO                  Int64
FI_POLIZA_MATRIZ       string
FI_CERTIFICADO         string
FF_CERTIFICADO         string
FI_ANEXO               string
FF_ANEXO               string
FI_DOCUMENTO           string
FF_DOCUMENTO           string
FI_VIGENCIA_ANT        string
FECHA_EXPE             string
TIPO_TRANSAC            Int64
FREC_PAGO               Int64
NRO_UNICO_TOMADOR       Int64
TIPO_IDENTIFI_TOM      string
NRO_IDENTIFI_TOM       string
INTERMEDIARIO_LIDE      Int64
CANAL                  string
SEGMENTO                Int64
VR_ASEG_POLIZA        float64
VR_PRIMA_MN_ORIG      float64
VR_PRIMA_PESOS        float64
VR_PRIMA_ANUAL        float64
COD_DEPARTAMENTO       string
COD_CIUDAD             string
ORIGEN                 string
LLAVE_DT  

In [68]:
inicio7 = time.time()

intermediarios_query = f"""SELECT DISTINCT COD_INTERMEDIARIO AS INTERMEDIARIO_LIDE
                                           ,COD_SUCURSAL AS SUCURSAL_PROD
                           FROM APOYO.DWH_INTERMEDIARIOS_TOTAL"""

intermediarios = pd.read_sql_query(intermediarios_query,cnxn,dtype={"INTERMEDIARIO_LIDE":"Int64","SUCURSAL_PROD":"Int64"})

fin7 = time.time()
print("intermediarios_query: " + str((fin7-inicio7)/float(60)) + " min")

intermediarios_query: 0.002581357955932617 min


In [69]:
base_pp_mes = pd.merge(base_pp_mes,intermediarios,on=["INTERMEDIARIO_LIDE"],how="left")
base_pp_mes.dtypes

PERIODO_CONTABLE        Int64
SSEGURO                 Int64
SUCURSAL_PROD_x        string
RAMO_PROD              string
POLIZA                 string
CERTIFICADO            string
DOCUMENTO              string
ANEXO                  string
RECIBO                  Int64
FI_POLIZA_MATRIZ       string
FI_CERTIFICADO         string
FF_CERTIFICADO         string
FI_ANEXO               string
FF_ANEXO               string
FI_DOCUMENTO           string
FF_DOCUMENTO           string
FI_VIGENCIA_ANT        string
FECHA_EXPE             string
TIPO_TRANSAC            Int64
FREC_PAGO               Int64
NRO_UNICO_TOMADOR       Int64
TIPO_IDENTIFI_TOM      string
NRO_IDENTIFI_TOM       string
INTERMEDIARIO_LIDE      Int64
CANAL                  string
SEGMENTO                Int64
VR_ASEG_POLIZA        float64
VR_PRIMA_MN_ORIG      float64
VR_PRIMA_PESOS        float64
VR_PRIMA_ANUAL        float64
COD_DEPARTAMENTO       string
COD_CIUDAD             string
ORIGEN                 string
LLAVE_DT  

In [70]:
base_pp_mes["SUCURSAL_PROD"] = (base_pp_mes["SUCURSAL_PROD_y"].combine_first(base_pp_mes["SUCURSAL_PROD_x"]))
base_pp_mes.head()

,PERIODO_CONTABLE,SSEGURO,SUCURSAL_PROD_x,RAMO_PROD,POLIZA,CERTIFICADO,DOCUMENTO,ANEXO,RECIBO,FI_POLIZA_MATRIZ,FI_CERTIFICADO,FF_CERTIFICADO,FI_ANEXO,FF_ANEXO,FI_DOCUMENTO,FF_DOCUMENTO,FI_VIGENCIA_ANT,FECHA_EXPE,TIPO_TRANSAC,FREC_PAGO,NRO_UNICO_TOMADOR,TIPO_IDENTIFI_TOM,NRO_IDENTIFI_TOM,INTERMEDIARIO_LIDE,CANAL,SEGMENTO,VR_ASEG_POLIZA,VR_PRIMA_MN_ORIG,VR_PRIMA_PESOS,VR_PRIMA_ANUAL,COD_DEPARTAMENTO,COD_CIUDAD,ORIGEN,LLAVE_DT,LLAVE_V,NRO_PLACA,NRO_CHASIS,COD_MARCA,COD_CLASE,COD_TIPO,PESO,ANN_MODELO,CILINDRAJE,CAPACI_OCUPANTES,IND_ACCESORIOS,VR_ASEG_ACCESORIOS,VR_ASEG_CASCO,TIPO_RIESGO,SUCURSAL_PROD_y,SUCURSAL_PROD
0,201605,857251,13,6031,100000,0,2,0,2554383,20150507,20160507,20170507,20160507.0,20170507.0,20160507,20170507.0,20150507,20160502,2,5,565704,C,37514526,13336,1,1,3.042188e+09,1146967.0,1146967.0,1146966.96,68,1,IAXIS,6031-100000-0-0,6031-100000-0-20160507-20170507,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,13,13
1,201605,857251,13,6031,100000,0,3,0,2627626,20150507,20160507,20170507,20160507.0,20170507.0,20160507,20170507.0,20150507,20160516,4,5,565704,C,37514526,13336,1,1,3.042188e+09,-1146967.0,-1146967.0,1146966.96,68,1,IAXIS,6031-100000-0-0,6031-100000-0-20160507-20170507,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,13,13
2,201605,839600,37,6031,100003,0,2,0,2466007,20150506,20160506,20170506,20160506.0,20170506.0,20160506,20170506.0,20150506,20160422,2,5,564368,C,41691171,37166,1,1,3.071188e+09,763230.0,763230.0,763229.55,11,1,IAXIS,6031-100003-0-0,6031-100003-0-20160506-20170506,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,221,221
3,201701,839600,37,6031,100003,0,4,1,4180114,20150506,20160506,20170506,20170107.0,20170506.0,20170107,20170506.0,20150506,20170111,8,5,564368,C,41691171,37166,1,1,3.071188e+09,109390.0,109390.0,771788.00,11,1,IAXIS,6031-100003-0-1,6031-100003-0-20160506-20170506,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,221,221
4,201705,839600,37,6031,100003,0,5,1,4908816,20150506,20170506,20180506,20170506.0,20180506.0,20170506,20180506.0,20160506,20170417,2,5,564368,C,41691171,37166,1,1,3.072433e+09,1188728.0,1188728.0,1188729.37,11,1,IAXIS,6031-100003-0-1,6031-100003-0-20170506-20180506,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,221,221


In [71]:
base_pp_mes = base_pp_mes.drop(["SUCURSAL_PROD_x","SUCURSAL_PROD_y"], axis = 1)
base_pp_mes.dtypes

PERIODO_CONTABLE        Int64
SSEGURO                 Int64
RAMO_PROD              string
POLIZA                 string
CERTIFICADO            string
DOCUMENTO              string
ANEXO                  string
RECIBO                  Int64
FI_POLIZA_MATRIZ       string
FI_CERTIFICADO         string
FF_CERTIFICADO         string
FI_ANEXO               string
FF_ANEXO               string
FI_DOCUMENTO           string
FF_DOCUMENTO           string
FI_VIGENCIA_ANT        string
FECHA_EXPE             string
TIPO_TRANSAC            Int64
FREC_PAGO               Int64
NRO_UNICO_TOMADOR       Int64
TIPO_IDENTIFI_TOM      string
NRO_IDENTIFI_TOM       string
INTERMEDIARIO_LIDE      Int64
CANAL                  string
SEGMENTO                Int64
VR_ASEG_POLIZA        float64
VR_PRIMA_MN_ORIG      float64
VR_PRIMA_PESOS        float64
VR_PRIMA_ANUAL        float64
COD_DEPARTAMENTO       string
COD_CIUDAD             string
ORIGEN                 string
LLAVE_DT               string
LLAVE_V   

In [72]:
#base_pp_mes.to_csv("base_pp_mes.csv")

In [ ]:
#BASE_PP_MOD_2 -> CONCATENACION HISTORICO (6 AÑOS) + PERIODO
#base_pp_mes.to_csv("base_pp_mod_2.csv")

In [ ]:
#CONVERT TO DATE 
base_1=base_pp_mes
base_1["DIF"] = np.where(base_1["ORIGEN"] == "AS400",base_1["FF_DOCUMENTO"]-base_1["FI_DOCUMENTO"],base_1["FF_ANEXO"]-base_1["FI_ANEXO"])
base_1.head()

In [ ]:
base_1.DIF = base_1.DIF.astype('string')
base_1.DIF = base_1.DIF.replace(' days','', regex=True)
base_1.DIF = base_1.DIF.astype('Int64')
base_1.dtypes

In [ ]:
#CAMBIAR TO NP WHERE
def tipo_transac(x):
    if ((x["TIPO_TRANSAC"] == 8)&(x["VR_PRIMA_MN_ORIG"]>0)&(x["DIF"] == 28)):
        return 9
    elif ((x["TIPO_TRANSAC"] == 8)&(x["VR_PRIMA_MN_ORIG"]>0)&(x["DIF"] ==29)):
        return 9
    elif ((x["TIPO_TRANSAC"] == 8)&(x["VR_PRIMA_MN_ORIG"]>0)&(x["DIF"] ==30)):
        return 9
    elif ((x["TIPO_TRANSAC"] == 8)&(x["VR_PRIMA_MN_ORIG"]>0)&(x["DIF"] ==30)):
        return 9
    elif ((x["TIPO_TRANSAC"] == 8)&(x["VR_PRIMA_MN_ORIG"]>0)&(x["DIF"] ==31)):
        return 9 
    elif ((x["TIPO_TRANSAC"] == 8)&(x["VR_PRIMA_MN_ORIG"]<0)&(x["DIF"] ==28)):
        return 4 
    elif ((x["TIPO_TRANSAC"] == 8)&(x["VR_PRIMA_MN_ORIG"]<0)&(x["DIF"] ==29)):
        return 4 
    elif ((x["TIPO_TRANSAC"] == 8)&(x["VR_PRIMA_MN_ORIG"]<0)&(x["DIF"] ==30)):
        return 4
    elif ((x["TIPO_TRANSAC"] == 8)&(x["VR_PRIMA_MN_ORIG"]<0)&(x["DIF"] ==31)):
        return 4
    else:
        np.nan
    

base_1["TIPO_TRANSAC_"] = base_1.apply(tipo_transac, axis=1)
base_1.head()

In [ ]:
base_1["TIPO_TRANSAC__"] = (base_1["TIPO_TRANSAC_"].combine_first(base_1["TIPO_TRANSAC"]))
base_1.head()

In [ ]:
base_1 = base_1.drop(["TIPO_TRANSAC_","TIPO_TRANSAC"], axis = 1)
base_1.rename(columns = {"TIPO_TRANSAC__":"TIPO_TRANSAC"}, inplace = True)
base_1.dtypes

In [ ]:
base_1.TIPO_TRANSAC = base_1.TIPO_TRANSAC.astype('Int64')

In [ ]:
base_1.FI_DOCUMENTO = pd.to_datetime(base_1["FI_DOCUMENTO"], format= '%Y-%m-%d',errors = "coerce")
base_1.FF_DOCUMENTO = pd.to_datetime(base_1["FF_DOCUMENTO"], format= '%Y-%m-%d',errors = "coerce")
base_1.FF_ANEXO = pd.to_datetime(base_1["FF_ANEXO"], format= '%Y-%m-%d',errors = "coerce")
base_1.FI_ANEXO = pd.to_datetime(base_1["FI_ANEXO"], format= '%Y-%m-%d',errors = "coerce")
base_1.FI_CERTIFICADO = pd.to_datetime(base_1["FI_CERTIFICADO"], format= '%Y-%m-%d',errors = "coerce")
base_1.FF_CERTIFICADO = pd.to_datetime(base_1["FF_CERTIFICADO"], format= '%Y-%m-%d',errors = "coerce")
base_1.FI_POLIZA_MATRIZ = pd.to_datetime(base_1["FI_POLIZA_MATRIZ"], format= '%Y-%m-%d',errors = "coerce")
base_1.FI_VIGENCIA_ANT = pd.to_datetime(base_1["FI_VIGENCIA_ANT"], format= '%Y-%m-%d',errors = "coerce")
base_1.FECHA_EXPE = pd.to_datetime(base_1["FECHA_EXPE"], format= '%Y-%m-%d',errors = "coerce")
base_1.head()

In [ ]:
autos = base_1.loc[:,["LLAVE_DT","NRO_PLACA","NRO_CHASIS","COD_MARCA","COD_CLASE","COD_TIPO","PESO","ANN_MODELO","CILINDRAJE","CAPACI_OCUPANTES","IND_ACCESORIOS","VR_ASEG_ACCESORIOS","VR_ASEG_CASCO"]].copy()
autos = autos.drop_duplicates()
autos.head()

In [ ]:
imagen = base_1.groupby("LLAVE_V").agg({"ANEXO":'max',"DOCUMENTO":'max'})
imagen.rename(columns = {"ANEXO":"ANEXO_MAX","DOCUMENTO":"DOCUMENTO_MAX"}, inplace = True)
imagen.head()

In [ ]:
base_1 = pd.merge(base_1,imagen,on=["LLAVE_V"],how="left")
base_1.dtypes

In [ ]:
base_1 = base_1[["PERIODO_CONTABLE","SSEGURO","SUCURSAL_PROD","RAMO_PROD","POLIZA","CERTIFICADO","DOCUMENTO","ANEXO","RECIBO""","FI_POLIZA_MATRIZ","FI_CERTIFICADO","FF_CERTIFICADO","FI_ANEXO","FF_ANEXO","FI_DOCUMENTO","FF_DOCUMENTO","FI_VIGENCIA_ANT","FECHA_EXPE","TIPO_TRANSAC","FREC_PAGO","NRO_UNICO_TOMADOR","TIPO_IDENTIFI_TOM","NRO_IDENTIFI_TOM","INTERMEDIARIO_LIDE","CANAL","SEGMENTO","VR_ASEG_POLIZA","VR_PRIMA_MN_ORIG","VR_PRIMA_PESOS","VR_PRIMA_ANUAL","COD_DEPARTAMENTO","COD_CIUDAD","LLAVE_DT","LLAVE_V","ORIGEN","TIPO_RIESGO"]]
base_1["IMAGEN"] = np.where(base_1["ORIGEN"] == "AS400","D","A")
base_1["LLAVE_DT_M"] = base_1["RAMO_PROD"] + "-" + base_1["POLIZA"] + "-" + base_1["CERTIFICADO"] + "-" + base_1["IMAGEN"]

In [ ]:
base_1 = pd.merge(base_1,autos,on=["LLAVE_DT"],how="left")
len(base_1)

In [ ]:
#REVISAR NRO_PLACA, es igual a LLENOS?
llenos_1 = base_1.loc[base_1["NRO_PLACA"] != '',:].copy()

In [ ]:
vacios_1 = base_1.loc[vacios_1["NRO_PLACA"].isna(),:].copy()

In [ ]:
vacios_2 = vacios_1.loc[:,["RAMO_PROD","POLIZA","CERTIFICADO","FI_CERTIFICADO","FF_CERTIFICADO","LLAVE_V"]].copy()

In [ ]:
llenos = base_1.loc[base_2["NRO_PLACA"].notnull(),["RAMO_PROD","POLIZA","CERTIFICADO","FI_CERTIFICADO","FF_CERTIFICADO","LLAVE_V","NRO_PLACA","NRO_CHASIS","COD_MARCA","COD_CLASE","COD_TIPO","PESO","ANN_MODELO","CILINDRAJE","CAPACI_OCUPANTES","IND_ACCESORIOS","VR_ASEG_ACCESORIOS","VR_ASEG_CASCO"]].copy()
llenos = llenos.drop_duplicates()
llenos.rename(columns = {"FI_CERTIFICADO":"FI","FF_CERTIFICADO":"FF"}, inplace = True)

In [ ]:
a_llenar_1 = pd.merge(vacios_2,llenos,on=["RAMO_PROD","POLIZA","CERTIFICADO"],how="left")
a_llenar_1.head()

In [ ]:
a_llenar_1["LLAVE_V"] = (a_llenar_1["LLAVE_V_x"].combine_first(a_llenar_1["LLAVE_V_y"]))
a_llenar_1 = a_llenar_1.drop(["LLAVE_V_x","LLAVE_V_y"],axis = 1)
a_llenar_1.head()

In [ ]:
a_llenar_1.FI_CERTIFICADO = pd.to_datetime(a_llenar_1["FI_CERTIFICADO"], format= '%Y-%m-%d',errors = "coerce")
a_llenar_1.FI = pd.to_datetime(a_llenar_1["FI"], format= '%Y-%m-%d',errors = "coerce")
a_llenar_1.head()

In [ ]:
a_llenar_1["DIF"] = a_llenar_1["FI_CERTIFICADO"] - a_llenar_1["FI"]
a_llenar_1 = a_llenar_1[a_llenar_1["NRO_PLACA"].notnull()]
a_llenar_1.drop_duplicates()
a_llenar_1.sort_values(["LLAVE_V","DIF"],ascending = (True,True))
a_llenar_1["RANK"] = a_llenar_1.groupby("LLAVE_V").cumcount() +1
a_llenar_1 = a_llenar_1[a_llenar_1["RANK"] == 1]
a_llenar_1 = a_llenar_1.drop(["RANK","DIF","FI","FF"],axis = 1)
a_llenar_1.dtypes
a_llenar_1.head()

In [ ]:
#SERVIRA GENERAR UN VACIOS_3
vacios_3 = pd.merge(vacios_1,a_llenar,on=["LLAVE_V"],how="left")
vacios_3.dtypes

In [ ]:
base_3_1 = pd.concat([llenos_1,vacios_3],ignore_index=True)
base_3_1.head()

In [ ]:
def ot(x):
    if ((x["TIPO_TRANSAC"] == 1)):
        return "A"
    elif ((x["TIPO_TRANSAC"] == 2)):
        return "B"
    elif ((x["TIPO_TRANSAC"] == 9)):
        return "C"
    elif ((x["TIPO_TRANSAC"] == 5)):
        return "D"
    elif ((x["TIPO_TRANSAC"] == 7)):
        return "E" 
    elif ((x["TIPO_TRANSAC"] == 8)):
        return "F" 
    elif ((x["TIPO_TRANSAC"] == 4)):
        return "G" 
    elif ((x["TIPO_TRANSAC"] == 12)):
        return "H"
    else:
        "I"
         
    

base_3_1["OT"] = base_3_1.apply(ot, axis=1)
base_3_1["PLACA_VIG"] = base_3_1["NRO_PLACA"] + "-" + base_3_1["FI_CERTIFICADO"] + "-" + base_3_1["FF_CERTIFICADO"]

In [ ]:
base_3_1 = base_3_1.sort_values(["PLACA_VIG","RAMO_PROD","POLIZA","CERTIFICADO","OT","PERIODO_CONTABLE"],ascending=(True,True,True,True,True,True))

In [ ]:
base_3_1["RANK"] = base_3_1.groupby("LLAVE_V").cumcount() +1
base_3_1 = base_3_1.drop([(base_3_1.TIPO_TRANSAC in < (7,8,9)) & (base_3_1.NUEVO_DOCUMENTO > 1)].index)
base_3_1["PN"] = np.where(base_3_1["TIPO_TRANSAC"] in (4,12),"N","P")

In [ ]:
base_3_1["N"] = np.where(base_3_1["PN"] == "N",1,0)
base_3_1["P"] = np.where(base_3_1["PN"] == "P",1,0)

In [ ]:
base_3_1["LLAVE_5"] = base_3_1["NRO_PLACA"] +"-"+ base_3_1["FI_CERTIFICADO"] +"-"+ base_3_1["FF_CERTIFICADO"] +"-"+ base_3_1["PN"]

In [ ]:
base_3_1 = base_3_1[base_3_1["TIPO_TRANSAC"] != ' ']

In [ ]:
cuenta = base_4_1.groupby("PLACA_VIG").agg({"P":'sum',"N":'max'})
cuenta.rename(columns = {"P":"POSITIVA","N":"NEGATIVA"}, inplace = True)

In [ ]:
base_3_1 = pd.merge(base_3_1,cuenta,on=["PLACA_VIG"],how="left")

In [ ]:
base_3_1 = pd.merge(base_3_1,aseg_dt,on=["LLAVE_DT"],how="left")

In [ ]:
base_3_1 = base_3_1.sort_values(["LLAVE_5","RAMO_PROD","POLIZA","CERTIFICADO","PERIODO_CONTABLE","OT"],ascending = (True,True,True,True,True,True))

In [ ]:
base_4 = base_4_2
base_4["RANK"] = base_4.groupby("LLAVE_5").cumcount() +1
base_4["DIF"] = np.where(base_4["RANK"] == 1, base_4["POSITIVA"] - base_4["NEGATIVA"],base_4["DIF"])
base_4 = base_4.drop([(base_4.DIF >0 ) & (base_4.PN == "N")].index)
base_4 = base_4.drop([(base_4.NRO_PLACA.isna() ) & (base_4.NRO_IDENTIFI_ASEG.isna())].index)
base_4 = base_4.drop(["POSITIVA","NEGATIVA","LLAVE_5","DIF"], axis = 1)

In [ ]:
base_4.sort_values(["ORIGEN","PLACA_VIG","PN","RAMO_PROD","POLIZA","CERTIFICADO","PERIODO_CONTABLE","RECIBO","OT"],ascending = (True,True,False,False,False,False,False,False,False))

In [ ]:
base_5_1 = base_5_1[base_5_1["RAMO_PROD"] in ('6031','6032','6033','6034','6035','6036','6037','6043','6048','6060','6061','AL','AU','AX','CT','PM','LM','MT','MU','ME','MW') & base_5_1["SUCURSAL_PROD"].isin([177,214])]
base_5_1["LLAVE_RIESGO"] = base_5_1["NRO_PLACA"] +"-"+ base_5_1["TIPO_IDENTIFI_ASEG"] +"-"+ base_5_1["NRO_IDENTIFI_ASEG"] 
base_5_1 = base_5_1[base_5_1["NRO_PLACA"] != "ZZZZ1"]

In [ ]:
dicc = {"ZONA":"string","SUCURSAL":"Int64"}

zonas_act = pd.read_csv("zonas_act.csv", sep = ';' , dtype = dicc)
zonas_act.reset_index(drop=True, inplace=True)
zonas_act.rename(columns = {"SUCURSAL":"SUCURSAL_PROD"}, inplace = True)
zonas_act.head()

In [ ]:
base_5_2 = pd.merge(base_5_1,zonas_act,on=["SUCURSAL_PROD"],how="left")
base_5_2.sort_values(["LLAVE_RIESGO","FI_CERTIFICADO","PN","RAMO_PROD","POLIZA","CERTIFICADO","FECHA_EXPE","RECIBO","ANEXO","OT"],ascending=(True,True,True,False,False,False,False,False,False,False,False))

In [ ]:
base_r = base_5_2
#lenght indicador marca_nr $20.
base_r = base_r.drop(["NUEVO_DOCUMENTO"], axis = 1)
base_r["ND"] = base_r.groupby("LLAVE_RIESGO").cumcount() +1
base_r["ANTERIOR"] = base_r["ND"] - 1

In [ ]:
def marca_nr(x):
    if ((x["ND"] == 1) & (x["PN"] == "P")):
        return "NUEVA"
    elif ((x["ND"] == 1) & (x["PN"] == "N")):
        return "NUEVA_CANCELADA"
     
base_r["MARCA_NR"] = base_r.apply(marca_nr, axis=1)

In [ ]:
sucursal_prod_not_list = [177,214]
ramo_prod_privado = ['6031','6032','6033','6034','6035','6036','6037','6043','6048','6060','6061','AL','AU','AX','CT','PM','LM','MT','MU','ME','MW']
ramo_prod_colectivo = ['AT','AW','PE','TP','TT','TS','TU','6041','6042','6045','6046','6047','6038','6039','LO','MO','PO','TO','LF','MF','900753']
not_ramo_prod_list = ['AT','AW','PE','TT','TS','6041','6042','6045','6046','6047','LO','MO','PO','TO','LF','MF','900753']
ramo_prod_banco = ['LO','MO','PO','TO','LF','MF','900753']

base_privado = base_4.loc[(~base_4.SUCURSAL_PROD.isin(sucursal_prod_not_list))&(pd.DataFrame(base_4.RAMO_PROD.tolist()).isin(ramo_prod_privado).any(axis=1).values)&(base_4.NRO_PLACA.values!='ZZZZ1'),:].copy()
base_privado = pd.merge(base_privado,zonas_act,on='',how='')
base_privado = base_privado.sort_values(['',''],ascending=)

base_colectivo = base_4.loc[(~base_4.SUCURSAL_PROD.isin(sucursal_prod_not_list))&(pd.DataFrame(base_4.RAMO_PROD.tolist()).isin(ramo_prod_colectivo).any(axis=1).values)&(base_4.NRO_PLACA.values!='ZZZZ1'),:].copy()
base_colectivo = base_colectivo.drop(base_colectivo[(pd.DataFrame(df.RAMO_PROD.tolist()).isin(not_ramo_prod_list).any(axis=1).values)&(base_colectivo.DOCUMENTO.values=='0')].index)
base_colectivo = pd.merge(base_colectivo,zonas_act,on='',how='')
base_colectivo = base_colectivo.sort_values(['',''],ascending=)

base_banco = base_4.loc[(base_4.SUCURSAL_PROD.values==214)&(pd.DataFrame(base_4.RAMO_PROD.tolist()).isin(ramo_prod_banco).any(axis=1).values)&(base_4.NRO_PLACA.values!='ZZZZ1'),:].copy()
base_banco = base_banco.drop(base_colectivo[(base_colectivo.TIPO_TRANSAC.values==7)&(base_colectivo.VR_PRIMA_ANUAL.values=='0')].index)
base_banco = pd.merge(base_banco,caratula_banco,on='',how='')
base_banco = base_banco.sort_values(['',''],ascending=)

base_coomeva = base_4.loc[(base_4.SUCURSAL_PROD.values==177)&(base_4.NRO_PLACA.values!='ZZZZ1'),:].copy()
base_coomeva = pd.merge(base_coomeva,zona_coomeva,on='',how='')
base_coomeva = base_coomeva.sort_values(['',''],ascending=)

#list_of_df = [base_privado,base_colectivo,base_banco,base_coomeva]

In [ ]:
#CICLO
bases = [base_privado,base_colectivo,base_banco,base_coomeva]
errores_bases = []
for base in bases:
    counter = 0
    
    base['ND'] = base.groupby('LLAVE_RIESGO').cumcount()+1
    base['ANTERIOR']=base['ND']-1
    base['MARCA_NR']=''
    base['MARCA_NR']=np.where((base.ND.values==1)&(base.PN.values=='P'),'NUEVA',base['MARCA_NR'])
    base['MARCA_NR']=np.where((base.ND.values==1)&(base.PN.values=='N'),'NUEVA_CANCELADA',base['MARCA_NR'])
    base['INDICADOR']=''
    base['M']=''
    while (len(base[base['MARCA_NR']==''])>0):
        counter += 1
        #ITERACION
        anterior = base.loc[base.ND.values==counter,['LLAVE_RIESGO','ND','FI_CERTIFICADO','FF_CERTIFICADO','MARCA_NR']].copy()
        anterior.rename(columns={'LLAVE_RIESGO':'LLR','FI_CERTIFICADO':'FI_1','FF_CERTIFICADO':'FF_1','MARCA_NR':'MARCA_1','ND':'ND_ANTERIOR'},inplace=True)

        base = pd.merge(base,anterior,left_on=['LLAVE_RIESGO','ANTERIOR'],right_on=['LLR','ND_ANTERIOR'],how='left')
        #regla1
        list_1=['NUEVA','NUEVA_CANCELADA']
        base['INDICADOR']=np.where((pd.DataFrame(base.MARCA_1.tolist()).isin(list_1).any(axis=1).values)&(base.PN.values=='N'),'NUEVA_CANCELADA',base['INDICADOR'])
        #regla2
        list_2=['RENOVADA','RENOVADA_CANCELADA']
        base['INDICADOR']=np.where((pd.DataFrame(base.MARCA_1.tolist()).isin(list_2).any(axis=1).values)&(base.PN.values=='N')&(base.INDICADOR.values=='')&(base.FI_CERTIFICADO.values==base.FI_1.values),'RENOVADA_CANCELADA',base['INDICADOR'])
        #regla3
        base['INDICADOR']=np.where((base.PN.values=='P')&(base.INDICADOR.values=='')&(base.FI_CERTIFICADO.values>=base.FF_1.values),'RENOVADA',base['INDICADOR'])
        #regla4
        base['INDICADOR']=np.where((base.PN.values=='P')&(base.INDICADOR.values=='')&(base.FI_CERTIFICADO.values>=base.FI_1.values)&(base.MARCA_1=='RENOVADA_CANCELADA'),'RENOVADA',base['INDICADOR'])
        #regla5
        base['INDICADOR']=np.where((base.PN.values=='P')&(base.INDICADOR.values=='')&(base.FI_CERTIFICADO.values<base.FF_1.values),'NUEVA',base['INDICADOR'])
        #regla6
        base['INDICADOR']=np.where((base.PN.values=='P')&(base.INDICADOR.values=='')&(base.FI_CERTIFICADO.values>base.FF_1.values),'NUEVA',base['INDICADOR'])
        #regla7
        base['INDICADOR']=np.where((base.PN.values=='N')&(base.INDICADOR.values=='')&(base.FI_CERTIFICADO.values!=base.FI_1.values)&(pd.DataFrame(base.MARCA_1.tolist()).isin(list_2).any(axis=1).values),'NUEVA_CANCELADA',base['INDICADOR'])
        #regla8
        base['MARCA_NR']=np.where((base.INDICADOR.values!='')&(base.ND.values==counter+1)&(base.MARCA_NR.values==''),base['INDICADOR'],base['MARCA_NR'])
        #regla9
        base['M']=np.where((base.ND.values==counter+1)&(base.MARCA_NR.values==''),1,base['M'])
        #regla10
        base['INDICADOR']=np.where((base.ND.values>counter+1),'',base['INDICADOR'])
        
        errores = base.loc[base.M.values==1,['FI_CERTIFICADO','FF_CERTIFICADO','FI_1','FF_1']].copy()
        errores.rename(columns={'FI_CERTIFICADO':'FI','FF_CERTIFICADO':'FF'},inplace=True)
        errores['DIF_I'] = ((errores['FI']-errores['FI_1']).dt.days)/365
        errores['DIF_F'] = ((errores['FF']-errores['FF_1']).dt.days)/365
        
        base = base.drop(['LLR','ND_ANTERIOR','FI_1','FF_1','MARCA_1'], axis = 1)
        
    #¡aplico append aca abajo o afuera del for?
    errores_base.append(errores)
    marca_f = base.loc[:,['LLAVE_RIESGO','ANTERIOR','MARCA_NR']].copy()
    marca_f.rename(columns = {'MARCA_NR':'MARCA_F'}, inplace = True)
    marca_f = marca_f.drop_duplicates()
    
    base = pd.merge(base,marca_f,left_on =['LLAVE_RIESGO','ND'],right_on=['LLAVE_RIESGO','ANTERIOR'],how='left')
    
    #AGREGAR WHERE IN LIST
    
    #pd.DataFrame(base_4.RAMO_PROD.tolist()).isin(ramo_prod_privado).any(axis=1).values
    
    base = base.drop(base[(base.MARCA_NR.values == 'RENOVADA_CANCELADA') & (base.MARCA_F.values == 'RENOVADA')].index)
    base = base.drop(base[(base.MARCA_NR.values == 'NUEVA_CANCELADA') & (base.MARCA_F.values == 'RENOVADA')].index)
    base['PER_INICIO'] = (base['FI_CERTIFICADO'].dt.year)*100 + (base['FI_CERTIFICADO'].dt.month)
    base["RETROACTIVO"] = np.where((((base['MARCA_NR'] == 'NUEVA')|(base['MARCA_NR'] == 'RENOVADA'))&(base['PERIODO_CONTABLE']>base['PER_INICIO'])),'SI','NO')
    base = base.drop(["MARCA_F"], axis = 1)
    
    #AGREGAR WHERE IN LIST
    retroactivo = base.loc[(base['MARCA_NR']=='NUEVA')|(base['MARCA_NR']=='RENOVADA'),['LLAVE_V','RETROACTIVO']].copy()
    retroactivo.rename(columns = {'RETROACTIVO':'RETRO'}, inplace = True)
    retroactivo = retroactivo.drop_duplicates()
    
    base = pd.merge(base,retroactivo,on='LLAVE_V',how='left')
    
#DESDE ACA COMIENZA 'PP_PRIVADO_MES_1'
base_privado = base_privado[base_privado['MARCA_NR']!='']
base_privado['RETROACTIVO']=np.where(base_privado.RETRO.values!='',base_privado['RETRO'],base_privado['RETROACTIVO'])
base_privado['FECHA_IN_CERT']=(base_privado['FI_CERTIFICADO'].dt.year)*100+(base_privado['FI_CERTIFICADO'].dt.month)
base_privado['FECHA_IN_MATR']=(base_privado['FI_POLIZA_MATRIZ'].dt.year)*100+(base_privado['FI_POLIZA_MATRIZ'].dt.month)
base_privado['CONTADOR']==1
base_privado['TIPO_POLIZA']=='I'
base_privado = base_privado.drop(["OT","PN","N","P","INDICADOR","ND","ANTERIOR","RETRO"], axis = 1)
#ELIMINAR CAMPOS CON NUEVOS NOMBRES QUE AUN NO SE DEFINEN, I.E. FEC_INI_POLIZA
base_privado = base_privado[['PERIODO_CONTABLE','SUCURSAL','RAMO','POLIZA','CERTIFICADO','RECIBO','ANEXO','FEC_INIC_POLIZA','FEC_INC_CERT','FEC_FIN_CERT','TIPO_TRANSACCION','COD_TRANSACCION','TIPO_POLIZA','VR_PRIMA_PESOS','VR_PRIMA_MON_ORG','PRIMA_ANUAL','VR_ASEG_ACCESORIOS','VR_ASEG_CASCO','PLACA RIESGO','RETROACTIVO','FECHA_IN_CERT','FECHA_IN_MATR','MARCA','ZONA','CONTADOR']]

#DESDE ACA COMIENZA 'PP_COLECTIVO_MES_1'
base_colectivo = base_colectivo[base_colectivo['MARCA_NR']!='']
base_colectivo['RETROACTIVO']=np.where(base_colectivo.RETRO.values!='',base_colectivo['RETRO'],base_colectivo['RETROACTIVO'])
base_colectivo['FECHA_IN_CERT']=(base_colectivo['FI_CERTIFICADO'].dt.year)*100+(base_colectivo['FI_CERTIFICADO'].dt.month)
base_colectivo['FECHA_IN_MATR']=(base_colectivo['FI_POLIZA_MATRIZ'].dt.year)*100+(base_colectivo['FI_POLIZA_MATRIZ'].dt.month)
base_colectivo['CONTADOR']==1
base_colectivo['TIPO_POLIZA']=='C'
base_colectivo = base_colectivo.drop(["OT","PN","N","P","INDICADOR","ND","ANTERIOR","RETRO"], axis = 1)
#ELIMINAR CAMPOS CON NUEVOS NOMBRES QUE AUN NO SE DEFINEN, I.E. FEC_INI_POLIZA
base_colectivo = base_colectivo[['PERIODO_CONTABLE','SUCURSAL','RAMO','POLIZA','CERTIFICADO','RECIBO','ANEXO','FEC_INIC_POLIZA','FEC_INC_CERT','FEC_FIN_CERT','TIPO_TRANSACCION','COD_TRANSACCION','TIPO_POLIZA','VR_PRIMA_PESOS','VR_PRIMA_MON_ORG','PRIMA_ANUAL','VR_ASEG_ACCESORIOS','VR_ASEG_CASCO','PLACA','RIESGO','RETROACTIVO','FECHA_IN_CERT','FECHA_IN_MATR','MARCA','ZONA','CONTADOR']]

#DESDE ACA COMIENZA 'PP_BANCO_MES_1'
base_banco = base_banco[base_banco['MARCA_NR']!='']
base_banco['FECHA_IN_CERT']=(base_banco['FI_CERTIFICADO'].dt.year)*100+(base_banco['FI_CERTIFICADO'].dt.month)
base_banco['FECHA_IN_MATR']=(base_banco['FI_POLIZA_MATRIZ'].dt.year)*100+(base_banco['FI_POLIZA_MATRIZ'].dt.month)
base_banco['CONTADOR']==1
base_banco['TIPO_POLIZA']=='C'
base_banco['DESC_AGRUP_TIPO']=='VACIO'
base_banco['ZONA']=='BANCO'
base_banco = base_banco.drop(["OT","PN","N","P","INDICADOR","ND","ANTERIOR","RETRO"], axis = 1)
#ELIMINAR CAMPOS CON NUEVOS NOMBRES QUE AUN NO SE DEFINEN, I.E. FEC_INI_POLIZA
base_banco = base_banco[['PERIODO_CONTABLE','SUCURSAL','RAMO','POLIZA','CERTIFICADO','RECIBO','ANEXO','FEC_INIC_POLIZA','FEC_INC_CERT','FEC_FIN_CERT','TIPO_TRANSACCION','COD_TRANSACCION','TIPO_POLIZA','VR_PRIMA_PESOS','VR_PRIMA_MON_ORG','PRIMA_ANUAL','VR_ASEG_ACCESORIOS','VR_ASEG_CASCO','PLACA','RIESGO','RETROACTIVO','FECHA_IN_CERT','FECHA_IN_MATR','MARCA','ZONA','CONTADOR','COBERTURA','DESCUENTO','TIPO','GRUPO','DESC_AGRUP_TIPO','DESCRIPCION','GRUPO_P','MARCA_P','DEPARTAMENTO']]

#DESDE ACA COMIENZA 'PP_COOMEVA_MES_1'
base_coomeva = base_coomeva[base_coomeva['MARCA_NR']!='']
base_coomeva['FECHA_IN_CERT']=(base_coomeva['FI_CERTIFICADO'].dt.year)*100+(base_coomeva['FI_CERTIFICADO'].dt.month)
base_coomeva['FECHA_IN_MATR']=(base_coomeva['FI_POLIZA_MATRIZ'].dt.year)*100+(base_coomeva['FI_POLIZA_MATRIZ'].dt.month)
base_coomeva['CONTADOR']==1
base_coomeva['TIPO_POLIZA']=='C'
base_coomeva = base_coomeva.drop(["OT","PN","N","P","INDICADOR","ND","ANTERIOR","RETRO"], axis = 1)
base_coomeva['ZONA']=np.where(base_coomeva.ZONA.values=='','CALI',base_coomeva['ZONA'])
#ELIMINAR CAMPOS CON NUEVOS NOMBRES QUE AUN NO SE DEFINEN, I.E. FEC_INI_POLIZA
base_coomeva = base_coomeva[['PERIODO_CONTABLE','SUCURSAL','RAMO','POLIZA','CERTIFICADO','RECIBO','ANEXO','FEC_INIC_POLIZA','FEC_INC_CERT','FEC_FIN_CERT','TIPO_TRANSACCION','COD_TRANSACCION','TIPO_POLIZA','VR_PRIMA_PESOS','VR_PRIMA_MON_ORG','PRIMA_ANUAL','VR_ASEG_ACCESORIOS','VR_ASEG_CASCO','PLACA','RIESGO','RETROACTIVO','FECHA_IN_CERT','FECHA_IN_MATR','MARCA','ZONA','CONTADOR']]

for base in bases:
    base['RAMO']=''
    base['RAMO']=np.where(base.RAMO_PROD.values=='6041','AW',base['RAMO'])
    base['RAMO']=np.where(base.RAMO_PROD.values=='6031','AU',base['RAMO'])
    base['RAMO']=np.where(base.RAMO_PROD.values=='6032','AX',base['RAMO'])
    base['RAMO']=np.where(base.RAMO_PROD.values=='6033','AL',base['RAMO'])
    base['RAMO']=np.where(base.RAMO_PROD.values=='6034','MT',base['RAMO'])
    base['RAMO']=np.where(base.RAMO_PROD.values=='6035','PM',base['RAMO'])
    base['RAMO']=np.where(base.RAMO_PROD.values=='6036','CT',base['RAMO'])
    base['RAMO']=np.where(base.RAMO_PROD.values=='6037','AU',base['RAMO'])
    base['RAMO']=np.where(base.RAMO_PROD.values=='6038','TI',base['RAMO'])
    base['RAMO']=np.where(base.RAMO_PROD.values=='6039','PI',base['RAMO'])
    base['RAMO']=np.where(base.RAMO_PROD.values=='6042','AW',base['RAMO'])
    
    base['RAMO']=np.where(base.RAMO_PROD.values=='6043','MW',base['RAMO'])
    base['RAMO']=np.where(base.RAMO_PROD.values=='6045','AT',base['RAMO'])
    base['RAMO']=np.where(base.RAMO_PROD.values=='6046','TT',base['RAMO'])
    base['RAMO']=np.where(base.RAMO_PROD.values=='6047','TP',base['RAMO'])
    base['RAMO']=np.where(base.RAMO_PROD.values=='6048','MW',base['RAMO'])
    base['RAMO']=np.where(base.RAMO_PROD.values=='6060','BV',base['RAMO'])
    base['RAMO']=np.where(base.RAMO_PROD.values=='6061','BV',base['RAMO'])
    base['RAMO']=np.where(base.RAMO_PROD.values=='',base['RAMO_PROD'],base['RAMO'])
    base['RECIBO']=np.where(base.RECIBO.values='',0,base['RECIBO'])
    base['COD_TRANSACCION']==0
    base.rename(columns = {'SUCURSAL_PROD':'SUCURSAL','FI_POLIZA_MATRIZ':'FEC_INIC_POLIZA','FI_CERTIFICADO':'FEC_INC_CERT','FF_CERTIFICADO':'FEC_FIN_CERT','TIPO_TRANSAC':'TIPO_TRANSACCION','VR_PRIMA_MN_ORIG':'VR_PRIMA_MON_ORG','VR_PRIMA_ANUAL':'PRIMA_ANUAL','NRO_PLACA':'PLACA','LLAVE_RIESGO':'RIESGO','MARCA_NR':'MARCA'}, inplace = True)
    
bases_privado_colectivo = [base_privado,base_colectivo]
bases_banco_coomeva = [base_banco,base_coomeva]

#%%time

type_dict_canal = {}

canal = pd.read_csv("canal.csv", index_col=0, dtype = type_dict_canal)
canal.head()

for base_pc in bases_privado_colectivo:
    base_pc = pd.merge(base_pc,canal[['CLASIFICACION_MOR','SUCURSAL_FUSION']],left_on ='SUCURSAL_PROD',right_on='SUCURSAL_FUSION',how='left')
    base_pc.rename(columns = {"CLASIFICACION MOR":"CANAL_2"}, inplace = True)
    base_pc.head()

ramo_prod_resumen_privado = ['6031','6032','6033','6035','6036','6037','6060','6061','AU','AL','AX','CT','PM','LM']

#SE AGREGA PERIODO ACA O EN EL COMIENZO?
resumen_privado = base_privado.loc[(base_privado.PERIODO_CONTABLE.values>=periodo-100),['PERIODO_CONTABLE','RAMO_PROD','MARCA_NR',]].copy()
resumen_privado['GRUPO'] = np.where(pd.DataFrame(base_privado.RAMO_PROD.tolist()).isin(ramo_prod_resumen_privado).any(axis=1).values,'AUTOS','MOTOS')
resumen_privado = resumen_privado.groupby(['PERIODO_CONTABLE','GRUPO','MARCA_NR']).agg({'RAMO_PROD':'count'})
resumen_privado.rename(columns={'RAMO_PROD':'CUENTA'},inplace=True)

ramo_prod_colectivo_livianos = ['6041','6042','6045','AT','AW','ET','LF']
ramo_prod_colectivo_pesados = ['6039','6047','TP','PE']
ramo_prod_colectivo_taxi = ['6038','6046','TT','TU','TS']

#SE AGREGA PERIODO ACA O EN EL COMIENZO?
resumen_colectivo = base_colectivo.loc[(base_colectivo.PERIODO_CONTABLE.values>=periodo-100),['PERIODO_CONTABLE','RAMO_PROD','MARCA_NR']].copy()
resumen_colectivo['GRUPO'] = ''
resumen_colectivo['GRUPO'] = np.where(pd.DataFrame(base_colectivo.RAMO_PROD.tolist()).isin(ramo_prod_colectivo_livianos).any(axis=1).values,'LIVIANOS',base_colectivo['GRUPO'])
resumen_colectivo['GRUPO'] = np.where(pd.DataFrame(base_colectivo.RAMO_PROD.tolist()).isin(ramo_prod_colectivo_pesados).any(axis=1).values,'PESADOS',base_colectivo['GRUPO'])
resumen_colectivo['GRUPO'] = np.where(pd.DataFrame(base_colectivo.RAMO_PROD.tolist()).isin(ramo_prod_colectivo_taxi).any(axis=1).values,'TAXI',base_colectivo['GRUPO'])
resumen_colectivo = resumen_colectivo.groupby(['PERIODO_CONTABLE','GRUPO','MARCA_NR']).agg({'RAMO_PROD':'count'})
resumen_colectivo.rename(columns={'RAMO_PROD':'CUENTA'},inplace=True)

#SE AGREGA PERIODO ACA O EN EL COMIENZO?
resumen_banco = base_banco.loc[(base_banco.PERIODO_CONTABLE.values>=periodo-100),['PERIODO_CONTABLE','RAMO_PROD','MARCA_NR']].copy()
resumen_banco = resumen_banco.groupby(['PERIODO_CONTABLE','GRUPO','MARCA_NR']).agg({'RAMO_PROD':'count'})
resumen_banco.rename(columns={'RAMO_PROD':'CUENTA'},inplace=True)

#SE AGREGA PERIODO ACA O EN EL COMIENZO?
resumen_coomeva = base_coomeva.loc[(base_coomeva.PERIODO_CONTABLE.values>=periodo-100),['PERIODO_CONTABLE','RAMO_PROD','MARCA_NR']].copy()
resumen_coomeva = resumen_coomeva.groupby(['PERIODO_CONTABLE','GRUPO','MARCA_NR']).agg({'RAMO_PROD':'count'})
resumen_coomeva.rename(columns={'RAMO_PROD':'CUENTA'},inplace=True)

ramo_prod_privado_r1 = ['6031','6032','6033','6034','6035','6036','6037','6043','6048','6060','6061','AL','AU','AX','CT','PM','LM','MT','MU','ME','MW']

privado_r1 = base_3_1.loc[((pd.DataFrame(base_3_1.RAMO_PROD.tolist()).isin(ramo_prod_privado_r1).any(axis=1).values)&(base_3_1.SUCURSAL_PROD!=177)),['RAMO_PROD','POLIZA','CERTIFICADO','FI_CERTIFICADO','FF_CERTIFICADO','NRO_PLACA','PLACA_VIG']].copy()
privado_r1 = privado_r1.drop_duplicates()

ramo_prod_colectivo_r1 = ['AT','AW','PE','TP','TT','TS','TU','6041','6042','6045','6046','6047','6038','6039','LO','MO','PO','TO','LF','MF','900753']
not_sucursal_prod_colectivo_r1 = [177,214]

colectivo_r1 = base_3_1.loc[((pd.DataFrame(base_3_1.RAMO_PROD.tolist()).isin(ramo_prod_colectivo_r1).any(axis=1).values)&(~base_3_1.SUCURSAL_PROD.isin(not_sucursal_prod_colectivo_r1))),['RAMO_PROD','POLIZA','CERTIFICADO','FI_CERTIFICADO','FF_CERTIFICADO','NRO_PLACA','PLACA_VIG']].copy()
colectivo_r1 = privado_r1.drop_duplicates()

banco_r1 = base_3_1.loc[base_3_1.SUCURSAL_PROD.values==214,['RAMO_PROD','POLIZA','CERTIFICADO','FI_CERTIFICADO','FF_CERTIFICADO','NRO_PLACA','PLACA_VIG']].copy()
banco_r1 = banco_r1.drop_duplicates()

coomeva_r1 = base_3_1.loc[base_3_1.SUCURSAL_PROD.values==177,['RAMO_PROD','POLIZA','CERTIFICADO','FI_CERTIFICADO','FF_CERTIFICADO','NRO_PLACA','PLACA_VIG']].copy()
coomeva_r1 = banco_r1.drop_duplicates()

marca_nr = ['NUEVA','RENOVADA']

''''for pp_1 in bases:
    pp_1 = base_r.loc[(pd.DataFrame(pp_1.RAMO_PROD.tolist()).isin(marca_nr).any(axis=1).values),['RAMO_PROD','POLIZA','CERTIFICADO','FI_CERTIFICADO','FF_CERTIFICADO','NRO_PLACA','PLACA_VIG','MARCA_NR','VR_PRIMA_ANUAL','VR_ASEG_CASCO']].copy()
    pp_1 = pp_1.drop_duplicates()
    pp_1 = pp_1.sort_values(['PLACA_VIG','MARCA_NR'],ascending = (False,False))
    pp_1['RANK'] = pp_1.groupby('PLACA_VIG').cumcount()+1
    pp_1 = pp_1.loc[pp_1.RANK.values==1,:]'''
    
privado = pd.merge(privado_r1,pp_1,on=['PLACA_VIG'],how="left")
colectivo = pd.merge(colectivo_r1,pp_1,on=['PLACA_VIG'],how="left")
banco = pd.merge(r1,pp_1,on=['PLACA_VIG'],how="left")
coomeva = pd.merge(r1,pp_1,on=['PLACA_VIG'],how="left")